# SelectQAOA: Regression Test Case Selection Using QAOAs
Regression testing is an important part of the software development process in software engineering. It is a practice aimed at identifying any regression, which is the emergence of new defects or issues in a software application following changes, enhancements, or updates made to the source code. In other words, regression testing focuses on how changes made to the software can affect the correct behavior of existing features. Regression testing is particularly important in agile software development environments, where changes are made frequently and rapidly. This practice helps ensure that the software remains stable and reliable as it evolves over time. Ideal regression testing would re-run all the available test cases of a given software system. However, in addition to being potentially very costly, this could even be impractical in some case. In this scenario, test case selection is one of the most widely investigated test suite optimization approaches.
Test case selection focuses on selecting a subset from an initial test suite to test software changes, i.e., to test whether unmodified parts of a program continue to work correctly after changes involving other parts. Furthermore, while in Test Case Minimization, not selecting a test case means permanently removing it; in Test Case Selection, the identified subset of the starting test suite is bounded to a specific execution of regression testing. Various techniques, such as Integer Programming, symbolic execution, data flow analysis, dependence graph-based methods, and flow graph-based approaches, can be employed to identify the modified portions of the software. Once test cases covering the unchanged program segments are pinpointed using a specific technique, an optimization algorithm (e.g., additional greedy, DIV-GA,
SelectQA, BootQA or SelectQAOA) can select a minimal set of these test cases based on certain testing criteria (e.g., branch coverage). The ultimate aim is to reduce the expenses associated with regression testing.

## Quantum Approximate Optimization Algorithm (QAOA)
The Quantum Approximate Optimization Algorithm (QAOA) is a quantum algorithm designed to tackle combinatorial optimization problems by combining classical and quantum computing techniques. It encodes the optimization problem into a Quantum Unconstrained Binary Optimization (QUBO) format and constructs a parameterized quantum circuit that alternates between applying the problem Hamiltonian, which represents the optimization goal, and a mixing Hamiltonian, which promotes exploration of the solution space. The parameters of this circuit are optimized using classical optimization methods to maximize the likelihood of measuring the optimal solution. Once the parameters are tuned, the circuit is executed on a quantum device or simulator to get candidate solutions, making QAOA particularly suitable for NP-hard problems like graph partitioning and maximum cut. This hybrid approach highlights the potential of quantum computing to provide advantages in solving complex optimization challenges.

In [1]:
#this cell contains all the imports needed by the pipeline
#to run it on the browser: jupyter notebook --NotebookApp.iopub_data_rate_limit=1.0e10
import os
import json
import time

import pandas as pd
import matplotlib.pyplot as plt
import random
import numpy as np
import statistics
import ast
import csv
from scipy.stats import mannwhitneyu, shapiro
from scipy.optimize import minimize

from qiskit_optimization import QuadraticProgram
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit_algorithms.optimizers import COBYLA
from qiskit.primitives import BackendSampler
from qiskit_algorithms import QAOA
from qiskit_algorithms.utils import algorithm_globals
from qiskit_aer import Aer, AerSimulator
from qiskit_aer.noise import NoiseModel, depolarizing_error
from qiskit_ibm_runtime.fake_provider import FakeVigoV2
from qiskit_optimization.converters import QuadraticProgramToQubo
from qiskit.circuit.library import QAOAAnsatz

from mitiq import zne
from mitiq.zne.inference import RichardsonFactory
from mitiq.zne.scaling import fold_global

from sklearn.preprocessing import StandardScaler
from scipy.cluster.hierarchy import linkage, fcluster
from collections import defaultdict

## Multi-Objective SelectQAOA
The first part of this work aims to compare SelectQAOA to the Multi-Objective DIV-GA, Additional Greedy, and SelectQA in terms of effectiveness and efficiency.
So, we will first focus on the implementation of a Multi-Objective version of SelectQAOA.

### The dataset
To experiment the performance of the proposed solution by this work and to compare its results to those of the classical state-of-the-art solutions, four public programs have been downloaded from the SIR website. SIR is a repository of software-related artifacts meant to support rigorous controlled experimentation with program analysis and software testing techniques, and education in controlled experimentation. 

### Chosen SIR Programs
The programs that will be used for experimentation have all been written in C and are:
- flex (a program that generates a lexical analysis program, based on regular expressions and C statement contained in one or more input files);
- grep (a useful program to search for matching patterns in a file);
- gzip (a program that substitutes a file, generally text files or web pages, with their compressed version)
- sed (a powerful program for stream text editing).

### Necessary information
The information needed by the quantum algorithm to work on every one of the four programs is:
- a fault matrix: it indicates whether a precise test case already found, during previous execution, a bug in the source code or not;
- execution cost: it indicates the execution cost of any test case of the suite;
- statement coverage: it indicates statement coverage information for every test case. 

All this information has been gathered through previous experimentation on the four programs mentioned above and written in files organized in the SIR_Programs folder. So, the first goal of the project will be gathering data from these files for computational purposes.

In [2]:
#this cell contains all variable definitions that will be useful throughout the entire project
sir_programs = ["flex","grep","gzip","sed"]
sir_programs_tests_number = {"flex":567,"grep":806,"gzip":214,"sed":360}
sir_programs_end_lines = {"flex":14192,"grep":13281,"gzip":6701,"sed":7118}
sir_programs_rep_values = {"flex":1,"grep":16,"gzip":1,"sed":1}
alpha = 0.5
experiments = 10

In [3]:
def json_keys_to_int(d):
    """This method correctly converts the data"""
    if isinstance(d, dict):
        return {int(k) if k.isdigit() else k: json_keys_to_int(v) for k, v in d.items()}
    elif isinstance(d, list):
        return [json_keys_to_int(i) for i in d]
    else:
        return d


with open("datasets/sir_programs/executed_lines_test_by_test.json", "r") as file:
    #dictionary that, for each sir program, associates at each LINE of that program the LIST of TESTS COVERING it
    executed_lines_test_by_test = json_keys_to_int(json.load(file)) #{program1:{line:[tci,tcj,...,tck],line2:...}
with open("datasets/sir_programs/faults_dictionary.json", "r") as file:
    #dictionary that associates at each SIR PROGRAM the LIST of PAST FAULT COVERAGE VALUES ORDERED BY TEST 
    faults_dictionary = json.load(file) #{program1:[fault_cov_tc1,fault_cov_tc2,...,fault_cov_tcn],program2:...}
with open("datasets/sir_programs/test_coverage_line_by_line.json", "r") as file:
    #dictionary that, for each sir program, associates at each TEST of that program the LIST of LINES COVERED by it
    test_coverage_line_by_line = json_keys_to_int(json.load(file)) #{program1:{tc1:[linei,linej,...,linek],tc2:...}
with open("datasets/sir_programs/test_cases_costs.json", "r") as file:
    #dictionary that, for each sir program, associates at each TEST its EXECUTION COST
    test_cases_costs = json_keys_to_int(json.load(file)) #{program1:{tc1:ex_cost1,tc2:ex_cost2,...,tcn:ex_costn},program2:...}
with open("datasets/sir_programs/total_program_lines.json", "r") as file:
    #dictionary which associates at each SIR PROGRAM its size in terms of the NUMBER OF ITS LINES
    total_program_lines = json.load(file) #{program1:tot_lines_program1,program2:tot_lines_program2,program3:...}

## Additional Greedy
One of the classical strategies generally used to solve the test case selection problem is the "Additional Greedy" approach. This strategy is applied using the standard weighted sum approach to conflate all the objectives to optimize the weighted sum of code coverage per unit time and fault coverage per unit time. This way, a list of non dominated solutions is built incrementally.

In [13]:
def lines_to_cover(sir_program):
    """This function is needed to know which are the lines that have to be covered."""
    lines_to_cover = set()
    
    for covered_lines in test_coverage_line_by_line[sir_program].values():
        for covered_line in covered_lines:
            lines_to_cover.add(covered_line)
            
    return lines_to_cover

def additional_greedy(sir_program):
    """This functions implements the Additional Greedy algorithm. It incrementally builds a set of non dominated solutions, choosing each time the test case that minimizes the function described earlier."""
    c = set()
    p = lines_to_cover(sir_program)
    s = []
    pareto_front = []
    test_cases_already_selected = []
    while len(c) < len(p):
        weighted_sum = []
        for test_case in range(0,sir_programs_tests_number[sir_program]):
            if test_case not in test_cases_already_selected and test_case in test_coverage_line_by_line[sir_program].keys():
                max_cost = max(test_cases_costs[sir_program].values())
                added_coverage = [line for line in test_coverage_line_by_line[sir_program][test_case] if line not in c]
                try:
                    weighted_sum[test_case]=((0.5 * (len(added_coverage)/len(test_coverage_line_by_line[sir_program][test_case]))) + (0.5 * faults_dictionary[sir_program][test_case]))/(test_cases_costs[sir_program][test_case]/max_cost)
                except:
                    pass
        # take the test case that minimizes the function
        best_test_case = min(weighted_sum,key=weighted_sum.get)
        test_cases_already_selected.append(best_test_case)
        for covered_line in test_coverage_line_by_line[sir_program][best_test_case]:
            # update the count of covered lines
            c.add(covered_line)
        s.append(best_test_case)
        # incremental building of the pareto frontier
        pareto_front.append(s.copy())
    
    return pareto_front

# running the greedy algorithm and saving the resulting solutions with execution times into .json files 
for sir_program in sir_programs:
    with open(f"results/add-greedy/{sir_program}_data.json", "w") as file:
        json_data = []
        start = time.time()
        json_data["pareto_front"] = additional_greedy(sir_program)
        end = time.time()
        json_data["resolution_time(ms)"] = (end-start)*1000
        json.dump(json_data,file)


AttributeError: 'list' object has no attribute 'get'

## Problem Decomposition with Clustering
The qubit availability is the most significant limitation of the currently available quantum computers.
Therefore, the test suites to optimize are decomposed into smaller sub-suites solvable by QAOA simulators using clustering techniques. Applying clustering allows for preserving the similarities between test cases in sub-suites, and since similar test cases tend to have redundant coverage, the optimization process will facilitate useless duplications. Also, the similarities between test cases of the same clusters imply a diversified representation of the initial suite, facilitating targeted balancing of the goal while building a diversified final sub-suite.

## QUBO Problems
A Quadratic Unconstrained Binary Optimization (QUBO) problem is a type of mathematical problem where we seek to find the best combination of binary values (0 or 1) for a set of variables to minimize or maximize an objective function. In other words, we are looking for the optimal solution among all possible variable combinations that satisfies certain constraints and makes the objective function as small as possible.

Weights for constraints (or penalty coefficients) are used in QUBO problems to assign a numerical value to the constraints and influence the optimization process. These weights are important because they allow for managing the priority and relative importance of constraints within the optimization problem.

In [5]:
def num_of_covered_lines(sir_program,test_cases):
    """This method returns the number of covered lines (no redundancy)"""
    covered_lines = set()
    
    for test_case in test_cases:
        try:
            for covered_line in test_coverage_line_by_line[sir_program][test_case]:
                covered_lines.add(covered_line)
        except:
            continue
    
    return len(covered_lines)

clusters_dictionary = dict()

for sir_program in sir_programs:
    tot_test_cases = sir_programs_tests_number[sir_program]
    
    # from {..., test_case_i : [cov_stmts], ...} to [..., #_stmt_cov_i, ...]
    test_cases_stmt_cov = []
    for test_case in test_coverage_line_by_line[sir_program].keys():
        test_cases_stmt_cov.append(len(test_coverage_line_by_line[sir_program][test_case]))
    suite_stmt_cov = sum(test_cases_stmt_cov)
    
    # Normalize data
    data = np.column_stack((list(test_cases_costs[sir_program].values()),faults_dictionary[sir_program],test_cases_stmt_cov))
    scaler = StandardScaler()
    normalized_data = scaler.fit_transform(data)

    num_clusters = 50
        
    max_cluster_dim = 5
    
    # Step 2: Perform K-Means Clustering
    start = time.time()
    linkage_matrix = linkage(normalized_data, method='ward')
    clusters = fcluster(linkage_matrix, t=num_clusters, criterion='maxclust')
    
    # Organize test cases by cluster
    clustered_data = defaultdict(list)
    for idx, cluster_id in enumerate(clusters):
        clustered_data[cluster_id].append(idx)
    
    # Process clusters to ensure none exceed max_cluster_dim
    new_cluster_id = max(clustered_data.keys()) + 1  # Start new IDs after existing ones
    to_add = []  # Collect new smaller clusters
    
    for cluster_id, elements in list(clustered_data.items()):  # Avoid modifying dict during iteration
        if len(elements) > max_cluster_dim:
            num_splits = -(-len(elements) // max_cluster_dim)  # Ceiling division to get the required number of splits
            split_size = -(-len(elements) // num_splits)  # Recalculate to distribute elements evenly
            
            # Split while keeping sizes balanced
            parts = [elements[i:i + split_size] for i in range(0, len(elements), split_size)]
    
            # Ensure all new clusters are within max_cluster_dim
            for part in parts:
                if len(part) > max_cluster_dim:
                    raise ValueError(f"A split cluster still exceeds max_cluster_dim ({len(part)} > {max_cluster_dim})!")
    
            # Add new parts to the new clusters
            to_add.extend(parts)
    
            # Remove original large cluster
            del clustered_data[cluster_id]
    
    # Assign new IDs to split parts
    for part in to_add:
        if part:  # Only add if the part is non-empty
            clustered_data[new_cluster_id] = part
            new_cluster_id += 1
    end = time.time()
    print("SelectQAOA Decomposition Time(ms): " + str((end-start)*1000))
    
    clusters_dictionary[sir_program] = clustered_data
        
    # Step 3: Calculate the metrics for each cluster and validate
    cluster_metrics = {}
    for cluster_id in clustered_data.keys():
        tot_cluster_exec_cost = 0
        tot_cluster_past_fault_cov = 0
        tot_cluster_stmt_cov = 0
        for test_case in clustered_data[cluster_id]:
            tot_cluster_exec_cost += test_cases_costs[sir_program][test_case]
            tot_cluster_past_fault_cov += faults_dictionary[sir_program][test_case]
        tot_cluster_past_fault_cov = tot_cluster_past_fault_cov/tot_test_cases
        tot_cluster_stmt_cov = num_of_covered_lines(sir_program,clustered_data[cluster_id])/total_program_lines[sir_program]
        cluster_metrics[cluster_id] = {
            "tot_exec_cost": tot_cluster_exec_cost,
            "tot_past_fault_cov": tot_cluster_past_fault_cov,
            "tot_stmt_cov": tot_cluster_stmt_cov  # Avg stmt coverage per test case in cluster
        }
        print(f"Cluster {cluster_id + 1} metrics:")
        print(f"Test Cases: {clustered_data[cluster_id]}")
        print(f" - Num. Test Cases: {len(clustered_data[cluster_id]):.2f}")
        print(f" - Execution Cost: {tot_cluster_exec_cost:.2f}")
        print(f" - Past Fault Coverage (%): {tot_cluster_past_fault_cov}")
        print(f" - Statement Coverage (%): {tot_cluster_stmt_cov:.2f}\n")
    
    for cluster_id in clustered_data.keys():
        print("Test cases of cluster " + str(cluster_id) + ": " + str(len(clustered_data[cluster_id])))
    
    print("======================================================================================")
    
    print("Program Name: " + sir_program)
    
    for cluster_id in clustered_data.keys():
        if len(clustered_data[cluster_id]) > max_cluster_dim:
            print("Test cases of cluster " + str(cluster_id) + ": " + str(len(clustered_data[cluster_id])))
    
    # Plotting the clusters in 3D space
    fig = plt.figure(figsize=(10, 7))
    ax = fig.add_subplot(111, projection='3d')
    
    # Extracting data for plotting
    exec_costs = np.array(list(test_cases_costs[sir_program].values()))
    fault_covs = np.array(faults_dictionary[sir_program])
    stmt_covs = np.array(test_cases_stmt_cov)
    
    # Plot each cluster with a different color
    colors = plt.cm.get_cmap("tab10", num_clusters)  # A colormap with as many colors as clusters
    for cluster_id in clustered_data.keys():
        cluster_indices = clustered_data[cluster_id]
        
        # Plot each cluster's points
        ax.scatter(
            exec_costs[cluster_indices], 
            fault_covs[cluster_indices], 
            stmt_covs[cluster_indices], 
            color=colors(cluster_id), 
            label=f"Cluster {cluster_id + 1}"
        )
    
    # Label the axes
    ax.set_xlabel("Execution Cost")
    ax.set_ylabel("Past Fault Coverage")
    ax.set_zlabel("Statement Coverage")
    ax.legend()
    ax.set_title("Test Case Clustering Visualization")
    
    # Display the plot
    plt.show()
    
print(clusters_dictionary)

SelectQAOA Decomposition Time(ms): 3.2248497009277344


IndexError: list assignment index out of range

In [6]:
def make_linear_terms(sir_program, cluster_test_cases, alpha):
    """Making the linear terms of QUBO"""
    max_cost = max(test_cases_costs[sir_program].values())
    
    estimated_costs = []

    #linear coefficients, that are the diagonal of the matrix encoding the QUBO
    for test_case in cluster_test_cases:
        estimated_costs.append((alpha * (test_cases_costs[sir_program][test_case]/max_cost)) - (1 - alpha) * faults_dictionary[sir_program][test_case])
    
    return np.array(estimated_costs)

def make_quadratic_terms(sir_program, variables, cluster_test_cases, linear_terms, penalty):
    """Making the quadratic terms of QUBO"""
    quadratic_terms = {}
    
    #k is a stmt
    for k in executed_lines_test_by_test[sir_program].keys():
        #k_test_cases is the list of test cases covering k
        k_test_cases = executed_lines_test_by_test[sir_program][k]
        for i in k_test_cases:
            if i not in cluster_test_cases or i not in variables:
                continue
            for j in k_test_cases:
                if j not in cluster_test_cases or j not in variables:
                    continue
                if i < j:
                    linear_terms[variables.index(i)] -= penalty
                    try:
                        quadratic_terms[variables.index(i),variables.index(j)] += 2 * penalty
                    except:
                        quadratic_terms[variables.index(i),variables.index(j)] = 2 * penalty
    
    return quadratic_terms

In [ ]:
def create_QUBO_problem(linear_terms, quadratic_terms):
    """This function is the one that has to encode the QUBO problem that QAOA will have to solve. The QUBO problem specifies the optimization to solve and a quadratic binary unconstrained problem"""
    qubo = QuadraticProgram()
    
    for i in range(0,len(linear_terms)):
        qubo.binary_var('x%s' % (i))

    qubo.minimize(linear=linear_terms,quadratic=quadratic_terms)

    return qubo


In [ ]:
penalties_dictionary = {"flex":None,"grep":None,"gzip":None,"sed":None}

#to get a QUBO problem from a quadratic problem with constraints, we have to insert those constraints into the Hamiltonian to solve (which is the one encoded by the QUBO problem). When we insert constraint into the Hamiltonian, we have to specify also penalties
for sir_program in sir_programs:
    max_penalty = 0
    max_cost = max(test_cases_costs[sir_program].values())
    for i in range(sir_programs_tests_number[sir_program]):
        cost = (alpha * (test_cases_costs[sir_program][i]/max_cost)) - ((1 - alpha) * faults_dictionary[sir_program][i])
        if cost > max_penalty:
            max_penalty = cost
    penalties_dictionary[sir_program] = max_penalty + 1

In [ ]:
qubos_dictionary = {"flex":[],"grep":[],"gzip":[],"sed":[]}
#make a dictionary that saves, for each program, the correspondent QUBO
for sir_program in sir_programs:
    print("SIR Program:\n")
    for cluster_id in clusters_dictionary[sir_program]:
        print("Cluster ID: " + str(cluster_id))
        variables = []
        for idx in range(0, len(clusters_dictionary[sir_program][cluster_id])):
            variables.append(idx)
        linear_terms = make_linear_terms(sir_program, clusters_dictionary[sir_program][cluster_id], alpha)
        quadratic_terms = make_quadratic_terms(sir_program, variables, clusters_dictionary[sir_program][cluster_id], linear_terms, penalties_dictionary[sir_program])
        qubo = create_QUBO_problem(linear_terms, quadratic_terms)
        qubos_dictionary[sir_program].append(qubo)
        print(qubo)
        print("/--/--/--/--/--/--/--/--/--/--/--/--/--/--/--/--/--/--/--/--/--/--/--/--/--/--/--/--/")
    print("======================================================================================")

In [ ]:
def covered_lines(sir_program,test_cases_list):
    """Number of covered lines (no redundancy)"""
    covered_lines = set()
    
    for test_case in test_cases_list:
        try:
            for covered_line in test_coverage_line_by_line[sir_program][test_case]:
                covered_lines.add(covered_line)
        except:
            continue
    
    return len(covered_lines)

def build_pareto_front(sir_program,selected_tests):
    """This method builds the pareto front additionally from a sub test suite solution"""
    pareto_front = []
    max_fault_coverage = 0
    max_stmt_coverage = 0
    
    for index in range(1,len(selected_tests)+1):
        #exract the first index selected tests
        candidate_solution = selected_tests[:index]
        candidate_solution_fault_coverage = 0
        candidate_solution_stmt_coverage = 0
        for selected_test in candidate_solution:
            candidate_solution_fault_coverage += faults_dictionary[sir_program][selected_test]
            candidate_solution_stmt_coverage += covered_lines(sir_program,candidate_solution)
        #if the actual pareto front dominates the candidate solution, get to the next candidate
        if max_fault_coverage >= candidate_solution_fault_coverage and max_stmt_coverage >= candidate_solution_stmt_coverage:
            continue
        #eventually update the pareto front information
        if candidate_solution_stmt_coverage > max_stmt_coverage:
            max_stmt_coverage = candidate_solution_stmt_coverage
        if candidate_solution_fault_coverage > max_fault_coverage:
            max_fault_coverage = candidate_solution_fault_coverage
        #add the candidate solution to the pareto front
        pareto_front.append(candidate_solution)
    
    return pareto_front

## Optimal Multi-Objective SelectQAOA Depth

First, we want to analyze whether the depth of the circuit of QAOA affects SelectQAOA. To do this, we have executed 10 independent times SelectQAOA on an IDEAL QAOA simulator for different depth values (1, 2, 4, 8, and 16) for each of the examined SIR programs. Then, we applied the Kruskal-Wallis H-test, to see if SelectQAOA behaves differently with different depth values. Finally, we used the Vargha-Delaney's $\hat{A}_{12}$ effect size to estimate the differences between the observations and identify the best configuration for the depth parameter.


## Statevector Simulator
The **Statevector Simulator** in Qiskit Aer is a high-precision quantum circuit simulator that represents the full quantum state of a system. In particular, it is a literal state vector or density matrix in a linear algebra sense, to obtain an exact simulation with no involved noise that applies the necessary rounding when performing floating-point operations. It is useful for **simulating ideal quantum circuits** without noise, decoherence, or measurement effects.


In [ ]:
algorithm_globals.random_seed = 10598
backend = Aer.get_backend("statevector_simulator")

#I want to run the sampler 30 times to get different results for each sir program
for sir_program in sir_programs:
    qaoa_mes = QAOA(sampler=BackendSampler(backend=backend), optimizer=COBYLA(100), reps=sir_programs_rep_values[sir_program])
    qaoa = MinimumEigenOptimizer(qaoa_mes)  # using QAOA
    #the fronts will be saved into files
    print("SIR Program: " + sir_program)
    file_path = "results/selectqaoa/statevector_sim/" + sir_program + "-data.json"
    json_data = []
    response = None
    qpu_run_times = []
    pareto_fronts_building_times = []
    for i in range(experiments):
        final_selected_tests = []
        cluster_dict_index = 0
        for qubo in qubos_dictionary[sir_program]:
            print("QUBO Problem: " + str(qubo) + "\n Cluster Number: " + str(cluster_dict_index))
            print("Cluster's Test Cases: " +str(list(clusters_dictionary[sir_program].values())[cluster_dict_index]))
            #for each iteration get the result
            s = time.time()
            qaoa_result = qaoa.solve(qubo)
            print("RESULTS: " + str(qaoa_result))
            e = time.time()
            qpu_run_times.append((e - s) * 1000)
            #let's extract the selected tests
            variable_values = qaoa_result.x
            indexes_selected_tests = [index for index, value in enumerate(variable_values) if value == 1]
            print("Indexes of selected tests to convert. " + str(indexes_selected_tests))
            selected_tests = []
            for index in indexes_selected_tests:
                selected_tests.append(list(clusters_dictionary[sir_program].values())[cluster_dict_index][index])
            print("Selected tests: " + str(selected_tests))
            print("Experiment Number: " + str(i))
            cluster_dict_index += 1
            for selected_test in selected_tests:
                if selected_test not in final_selected_tests:
                    final_selected_tests.append(selected_test)
        i+=1
        #now we have to build the pareto front
        print("Final Selected Test Cases: " + str(final_selected_tests))
        print("Length of the final list of selected test cases: " + str(len(final_selected_tests)))
        start = time.time()
        pareto_front = build_pareto_front(sir_program, final_selected_tests)
        end = time.time()
        json_data["pareto_front_" + str(i)] = pareto_front
        pareto_front_building_time = (end - start) * 1000
        pareto_fronts_building_times.append(pareto_front_building_time)

    #compute the average time needed for the construction of a pareto frontier and run time
    mean_qpu_run_time = statistics.mean(qpu_run_times)
    mean_pareto_fronts_building_time = statistics.mean(pareto_fronts_building_times)
    json_data["mean_qpu_run_time(ms)"] = mean_qpu_run_time
    json_data["stdev_qpu_run_time(ms)"] = statistics.stdev(qpu_run_times)
    json_data["all_qpu_run_times(ms)"] = qpu_run_times
    json_data["mean_pareto_fronts_building_time(ms)"] = mean_pareto_fronts_building_time

    with open(file_path, "w") as file:
        json.dump(json_data, file)


## Kruskal-Wallis H Test

The **Kruskal-Wallis H test** is a **non-parametric** statistical test used to determine whether **three or more independent groups** come from the same population. It is an extension of the **Mann-Whitney U test** for multiple groups and serves as a non-parametric alternative to **one-way ANOVA**.

- **Null Hypothesis (\(H_0\))**: The samples come from the same distribution (no significant difference between groups).
- **Alternative Hypothesis (\(H_A\))**: At least one group differs significantly from the others.

- A **p-value** is computed to assess statistical significance:
  - **If \( p > 0.05 \)** → Fail to reject \( H_0 \) → Groups may come from the same distribution.
  - **If \( p < 0.05 \)** → Reject \( H_0 \) → At least one group differs significantly.

## Vargha-Delaney's $\hat{A}_{12}$ Effect Size

The **Vargha-Delaney $\hat{A}_{12}$ effect size** is a non-parametric measure used to compare two distributions. It quantifies the probability that a randomly chosen value from one group ( X ) is **greater** than a randomly chosen value from another group ( Y ). It is particularly useful for analyzing differences in performance between two methods, conditions, or datasets.
- **$\hat{A}_{12}$ = 0.5**: Both groups have similar distributions.
- **$\hat{A}_{12}$ > 0.5**: Values in \( X \) tend to be larger than those in \( Y \).
- **$\hat{A}_{12}$ < 0.5**: Values in \( Y \) tend to be larger than those in \( X \).

In [ ]:
def total_cost(sir_program,solution):
    """Total execution cost of the sub suite"""
    solution_cost = 0
    
    for test_case in solution:
        try:
            solution_cost += test_cases_costs[sir_program][test_case]
        except:
            continue

    return solution_cost
        
def total_coverage(sir_program,solution):
    """Number of covered lines (no redundancy)"""
    covered_lines = set()
    
    for test_case in solution:
        try:
            for covered_line in test_coverage_line_by_line[sir_program][test_case]:
                covered_lines.add(covered_line)
        except:
            continue
    
    return len(covered_lines)
    
def total_faults(sir_program,solution):
    """Number of covered faults"""
    covered_faults = 0
    
    for test_case in solution:
        try:
            covered_faults += faults_dictionary[sir_program][test_case]
        except:
            continue
    
    return covered_faults

def pareto_dominance(tuple1, tuple2):
    """This method returns if tuple1 dominates tuple2"""
    # Check if all conditions are satisfied
    dominates = (
        (tuple2[0] <= tuple1[0]) and 
        (tuple2[1] >= tuple1[1]) and 
        (tuple2[2] >= tuple1[2])
    )
    
    # Check if at least one condition is strict
    strict = (
        (tuple2[0] < tuple1[0]) or 
        (tuple2[1] > tuple1[1]) or 
        (tuple2[2] > tuple1[2])
    )
    
    # Return 1 if the second tuple dominates the first, otherwise 0
    return 1 if dominates and strict else 0

In [ ]:
#here we make the same but for each of the fronts built by SelectQA and DIV-GA

#the following 2 lists will contain much tuples as solutions in each of the 10 pareto fronts
#each tuple represent the fitness value for each solution
json_data = {"flex":{"statevector_sim_1":[],"statevector_sim_2":[],"statevector_sim_4":[],"statevector_sim_8":[],"statevector_sim_16":[]},"grep":{"statevector_sim_1":[],"statevector_sim_2":[],"statevector_sim_4":[],"statevector_sim_8":[],"statevector_sim_16":[]},"gzip":{"statevector_sim_1":[],"statevector_sim_2":[],"statevector_sim_4":[],"statevector_sim_8":[],"statevector_sim_16":[]},"sed":{"statevector_sim_1":[],"statevector_sim_2":[],"statevector_sim_4":[],"statevector_sim_8":[],"statevector_sim_16":[]}}
normal_dist = True

for sir_program in sir_programs:
    qaoa_statevector_pareto_vectors_1 = []
    qaoa_statevector_pareto_vectors_2 = []
    qaoa_statevector_pareto_vectors_4 = []
    qaoa_statevector_pareto_vectors_8 = []
    qaoa_statevector_pareto_vectors_16 = []
    
    for index in range(0,30):
        # Load the JSON file
        with open('results/selectqaoa/statevector_sim/' + sir_program + '-data-rep-1.json', 'r') as file:
            pareto_fronts_json = json.load(file)
        
        qaoa_pareto_front = pareto_fronts_json['pareto_front_'+str(index+1)]
        #a single solution is a subset of the initial test suite
        for front_solution in qaoa_pareto_front:
            qaoa_statevector_pareto_vectors_1.append((total_cost(sir_program,front_solution),total_coverage(sir_program,front_solution),total_faults(sir_program,front_solution)))
        #print(len(qaoa_statevector_pareto_vectors_1))
        
        with open('results/selectqaoa/statevector_sim/' + sir_program + '-data-rep-2.json', 'r') as file:
            pareto_fronts_json = json.load(file)
        
        qaoa_pareto_front = pareto_fronts_json['pareto_front_'+str(index+1)]
        #a single solution is a subset of the initial test suite
        for front_solution in qaoa_pareto_front:
            qaoa_statevector_pareto_vectors_2.append((total_cost(sir_program,front_solution),total_coverage(sir_program,front_solution),total_faults(sir_program,front_solution)))
        #print(len(qaoa_statevector_pareto_vectors_2))
        
        with open('results/selectqaoa/statevector_sim/' + sir_program + '-data-rep-4.json', 'r') as file:
            pareto_fronts_json = json.load(file)
        
        qaoa_pareto_front = pareto_fronts_json['pareto_front_'+str(index+1)]
        #a single solution is a subset of the initial test suite
        for front_solution in qaoa_pareto_front:
            qaoa_statevector_pareto_vectors_4.append((total_cost(sir_program,front_solution),total_coverage(sir_program,front_solution),total_faults(sir_program,front_solution)))
        #print(len(qaoa_statevector_pareto_vectors_4))
        
        with open('results/selectqaoa/statevector_sim/' + sir_program + '-data-rep-8.json', 'r') as file:
            pareto_fronts_json = json.load(file)
        
        qaoa_pareto_front = pareto_fronts_json['pareto_front_'+str(index+1)]
        #a single solution is a subset of the initial test suite
        for front_solution in qaoa_pareto_front:
            qaoa_statevector_pareto_vectors_8.append((total_cost(sir_program,front_solution),total_coverage(sir_program,front_solution),total_faults(sir_program,front_solution)))
        #print(len(qaoa_statevector_pareto_vectors_8))
        
        with open('results/selectqaoa/statevector_sim/' + sir_program + '-data-rep-16.json', 'r') as file:
            pareto_fronts_json = json.load(file)
        
        qaoa_pareto_front = pareto_fronts_json['pareto_front_'+str(index+1)]
        #a single solution is a subset of the initial test suite
        for front_solution in qaoa_pareto_front:
            qaoa_statevector_pareto_vectors_16.append((total_cost(sir_program,front_solution),total_coverage(sir_program,front_solution),total_faults(sir_program,front_solution)))
        #print(len(qaoa_noise2_pareto_vectors))
    
    #once we have the pareto vectors from each pareto front obtained by each run, we extract 
    #just the solution not dominated by anyone else
    
    total_fronts = [qaoa_statevector_pareto_vectors_1,qaoa_statevector_pareto_vectors_2,qaoa_statevector_pareto_vectors_4,qaoa_statevector_pareto_vectors_8,qaoa_statevector_pareto_vectors_16]
    reference_pareto = []
    portions = [0,0,0,0,0]
    
    # get the reference frontier
    for index, front1 in enumerate(total_fronts):
        for front_solution1 in front1:
            is_dominated = 0
            other_fronts = total_fronts[:index] + total_fronts[index+1:]
            for front2 in other_fronts:
                for front_solution2 in front2:
                    if pareto_dominance(front_solution1,front_solution2):
                        is_dominated = 1
                        break
                if is_dominated:
                    break
            if not is_dominated:
                reference_pareto.append(front_solution1)
                portions[index] = portions[index] + 1
    
    print(portions)
    # For SelectQA and DIV-GA we want to compute, for each of the 10 iterations, how many of the solutions of the i-th pareto front were selected by the reference front
    
    qaoa_statevector_non_dominated_values_1 = []
    qaoa_statevector_non_dominated_values_2 = []
    qaoa_statevector_non_dominated_values_4 = []
    qaoa_statevector_non_dominated_values_8 = []
    qaoa_statevector_non_dominated_values_16 = []
    
    for index in range(0,30):
        #print("Iteratrion: " + str(index))
        
        qaoa_statevector_non_dominated_1 = 0
        qaoa_statevector_non_dominated_2 = 0
        qaoa_statevector_non_dominated_4 = 0
        qaoa_statevector_non_dominated_8 = 0
        qaoa_statevector_non_dominated_16 = 0
        
        # Load the JSON file
        with open('results/selectqaoa/statevector_sim/' + sir_program + '-data-rep-1.json', 'r') as file:
            pareto_fronts_json = json.load(file)
        
        qaoa_pareto_front = pareto_fronts_json['pareto_front_'+str(index+1)]
        for front_solution in qaoa_pareto_front:
            if (total_cost(sir_program,front_solution),total_coverage(sir_program,front_solution),total_faults(sir_program,front_solution)) in reference_pareto:
                qaoa_statevector_non_dominated_1 += 1
        
        qaoa_statevector_non_dominated_values_1.append(qaoa_statevector_non_dominated_1)
        #print(qaoa_statevector_non_dominated_1)
        
        with open('results/selectqaoa/statevector_sim/' + sir_program + '-data-rep-2.json', 'r') as file:
            pareto_fronts_json = json.load(file)
        
        qaoa_pareto_front = pareto_fronts_json['pareto_front_'+str(index+1)]
        for front_solution in qaoa_pareto_front:
            if (total_cost(sir_program,front_solution),total_coverage(sir_program,front_solution),total_faults(sir_program,front_solution)) in reference_pareto:
                qaoa_statevector_non_dominated_2 += 1
        
        qaoa_statevector_non_dominated_values_2.append(qaoa_statevector_non_dominated_2)
        #print(qaoa_statevector_non_dominated_1)
        
        with open('results/selectqaoa/statevector_sim/' + sir_program + '-data-rep-4.json', 'r') as file:
            pareto_fronts_json = json.load(file)
        
        qaoa_pareto_front = pareto_fronts_json['pareto_front_'+str(index+1)]
        for front_solution in qaoa_pareto_front:
            if (total_cost(sir_program,front_solution),total_coverage(sir_program,front_solution),total_faults(sir_program,front_solution)) in reference_pareto:
                qaoa_statevector_non_dominated_4 += 1
        
        qaoa_statevector_non_dominated_values_4.append(qaoa_statevector_non_dominated_4)
        #print(qaoa_statevector_non_dominated_4)
        
        with open('results/selectqaoa/statevector_sim/' + sir_program + '-data-rep-8.json', 'r') as file:
            pareto_fronts_json = json.load(file)
        
        qaoa_pareto_front = pareto_fronts_json['pareto_front_'+str(index+1)]
        for front_solution in qaoa_pareto_front:
            if (total_cost(sir_program,front_solution),total_coverage(sir_program,front_solution),total_faults(sir_program,front_solution)) in reference_pareto:
                qaoa_statevector_non_dominated_8 += 1
        
        qaoa_statevector_non_dominated_values_8.append(qaoa_statevector_non_dominated_8)
        #print(qaoa_statevector_non_dominated_4)
        
        with open('results/selectqaoa/statevector_sim/' + sir_program + '-data-rep-16.json', 'r') as file:
            pareto_fronts_json = json.load(file)
        
        qaoa_pareto_front = pareto_fronts_json['pareto_front_'+str(index+1)]
        for front_solution in qaoa_pareto_front:
            if (total_cost(sir_program,front_solution),total_coverage(sir_program,front_solution),total_faults(sir_program,front_solution)) in reference_pareto:
                qaoa_statevector_non_dominated_16 += 1
        
        qaoa_statevector_non_dominated_values_16.append(qaoa_statevector_non_dominated_16)
        #print(qaoa_statevector_non_dominated_8)
    
    print("SelectQAOA Non Dominated Values (Rep 1)")
    print(qaoa_statevector_non_dominated_values_1)
    print("SelectQAOA Non Dominated Mean (Rep 1)")
    print(statistics.mean(qaoa_statevector_non_dominated_values_1))
    print("SelectQAOA Non Dominated StDev (Rep 1)")
    print(statistics.stdev(qaoa_statevector_non_dominated_values_1))
    json_data[sir_program]["statevector_sim_1"] = qaoa_statevector_non_dominated_values_1
    
    print("SelectQAOA Non Dominated Values (Rep 2)")
    print(qaoa_statevector_non_dominated_values_2)
    print("SelectQAOA Non Dominated Mean (Rep 2)")
    print(statistics.mean(qaoa_statevector_non_dominated_values_2))
    print("SelectQAOA Non Dominated StDev (Rep 2)")
    print(statistics.stdev(qaoa_statevector_non_dominated_values_2))
    json_data[sir_program]["statevector_sim_2"] = qaoa_statevector_non_dominated_values_2
    
    print("SelectQAOA Non Dominated Values (Rep 4)")
    print(qaoa_statevector_non_dominated_values_4)
    print("SelectQAOA Non Dominated Mean (Rep 4)")
    print(statistics.mean(qaoa_statevector_non_dominated_values_4))
    print("SelectQAOA Non Dominated StDev (Rep 4)")
    print(statistics.stdev(qaoa_statevector_non_dominated_values_4))
    json_data[sir_program]["statevector_sim_4"] = qaoa_statevector_non_dominated_values_4
    
    print("SelectQAOA Non Dominated Values (Rep 8)")
    print(qaoa_statevector_non_dominated_values_8)
    print("SelectQAOA Non Dominated Mean (Rep 8)")
    print(statistics.mean(qaoa_statevector_non_dominated_values_8))
    print("SelectQAOA Non Dominated StDev (Rep 8)")
    print(statistics.stdev(qaoa_statevector_non_dominated_values_8))
    json_data[sir_program]["statevector_sim_18"] = qaoa_statevector_non_dominated_values_8
    
    print("SelectQAOA Non Dominated Values (Rep 16)")
    print(qaoa_statevector_non_dominated_values_16)
    print("SelectQAOA Non Dominated Mean (Rep 16)")
    print(statistics.mean(qaoa_statevector_non_dominated_values_16))
    print("SelectQAOA Non Dominated StDev (Rep 16)")
    print(statistics.stdev(qaoa_statevector_non_dominated_values_16))
    json_data[sir_program]["statevector_sim_16"] = qaoa_statevector_non_dominated_values_16
    
    print(json_data)
        
    if normal_dist:
        lists = [qaoa_statevector_non_dominated_values_1,qaoa_statevector_non_dominated_values_2,qaoa_statevector_non_dominated_values_4,qaoa_statevector_non_dominated_values_8,qaoa_statevector_non_dominated_values_16]

        # we are interested in knowing if the obtained sequences are normally distributed to decide what statistical test we should apply
        for i, list in enumerate(lists, start=1):
            stat, p_value = shapiro(list)
            #print(f"List {i}: Statistic = {stat:.5f}, P-value = {p_value:.5f}")
            
            # check if the sequence is normally distributed (using 0.05 as threshold)
            if p_value <= 0.05:
                #print(f"List {i} NOT seems to be normally distributed (p-value = {p_value:.5f})")
                normal_dist = False
                
if normal_dist:
    print("Values all normally distributed")
else:
    print("Values not normally distributed")

with open("statevector_multi_obj_frontiers_eval.json", "w") as file:
    json.dump(json_data, file)

In [ ]:
"""
Non dominated solutions comparing aer_sim executions

Flex (rep = 4 BEST) old: 1
list_r1 = [22, 1, 50, 49, 1, 51, 78, 1, 0, 344] m=59.7 stdev=103.69
list_r2 = [0, 13, 79, 1, 58, 132, 91, 110, 4, 9] m=49.7 stdev=50.52
list_r4 = [1, 128, 19, 64, 0, 184, 182, 104, 240, 220] m=114.2 stdev=90.81
list_r8 = [275, 0, 172, 99, 4, 48, 115, 4, 37, 52] m=80.6 stdev=87.99
list_r16 = [117, 22, 5, 0, 10, 111, 354, 31, 34, 4] m=68.8 stdev=108.90
H_test p_value > 0.05 -> no differences: BEST rep=1

Grep (rep = 16 BEST) old: 16
1. [7, 0, 1, 13, 4, 0, 4, 0, 1, 0] m=3 stdev=4.24
2. [25, 0, 37, 56, 0, 64, 1, 0, 2, 3] m=18.8 stdev=25.17
4. [125, 141, 2, 236, 102, 80, 1, 2, 0, 0] m=68.9 stdev=82.01
8. [16, 139, 0, 39, 0, 0, 75, 94, 0, 21] m=38.4 stdev=48.59
16. [414, 17, 329, 278, 207, 338, 9, 427, 137, 293] m=244.9 stdev=149.65
H_test p_value < 0.05 -> differences: BEST rep = 16 

Gzip (rep = 16 BEST) old: 1
1. [21, 0, 29, 8, 38, 28, 12, 0, 10, 25] m=17.1 stdev=12.99
2. [14, 0, 0, 12, 20, 2, 2, 1, 15, 87] m=15.3 stdev=26.25
4. [0, 0, 53, 4, 10, 5, 1, 2, 34, 1] m=11 stdev=17.95
8. [5, 53, 6, 6, 7, 10, 2, 9, 0, 6] m=10.4 stdev=15.25
16. [8, 17, 0, 125, 126, 1, 31, 88, 124, 1] m=52.1 stdev=56.56
H_test p_value > 0.05 -> no differences: BEST rep=1

Sed (rep = 16 BEST) old: 1
1. [3, 14, 45, 5, 8, 13, 13, 22, 127, 4] m=25.4 stdev=37.76
2. [35, 26, 4, 4, 18, 13, 25, 42, 3, 139] m=30.9 stdev=40.27
4. [6, 176, 6, 26, 42, 15, 5, 5, 191, 25] m=49.7 stdev=71.62
8. [3, 52, 8, 17, 5, 2, 65, 67, 4, 3] m=22.6 stdev=27.33
16. [28, 193, 161, 29, 36, 39, 68, 54, 64, 120] m=79.2 stdev=58.61
H_test p_value > 0.05 -> no differences: BEST rep=1
"""

## Multi-Objective SelectQAOA Execution
Once we know the optimal configuration of SelectQAOA for each depth value, we can make the executions of all the simulators with their optimal configurations.

## Aer Simulator
The Aer Simulator applies sampling noise to simulate the natural stochasticity property of quantum mechanics. No other perturbations are allowed, but the stochasticity itself is a significant obstacle for many optimization methods.

In [ ]:
#I want to run the sampler 30 times to get different results for each sir program
for sir_program in sir_programs:
    aer_sim = AerSimulator()
    algorithm_globals.random_seed = 10598
    qaoa_mes = QAOA(sampler=BackendSampler(backend=aer_sim), optimizer=COBYLA(100), reps=sir_programs_rep_values[sir_program])
    qaoa = MinimumEigenOptimizer(qaoa_mes)  # using QAOA
    #the fronts will be saved into files
    print("SIR Program: " + sir_program)
    file_path = "results/selectqaoa/aer_sim/" + sir_program + "-data.json"
    json_data = []
    response = None
    qpu_run_times = []
    pareto_fronts_building_times = []
    for i in range(experiments):
        final_selected_tests = []
        cluster_dict_index = 0
        for qubo in qubos_dictionary[sir_program]:
            print("QUBO Problem: " + str(qubo) + "\n Cluster Number: " + str(cluster_dict_index))
            print("Cluster's Test Cases: " +str(list(clusters_dictionary[sir_program].values())[cluster_dict_index]))
            #for each iteration get the result
            s = time.time()
            qaoa_result = qaoa.solve(qubo)
            print("RESULTS: " + str(qaoa_result))
            e = time.time()
            qpu_run_times.append((e - s) * 1000)
            #let's extract the selected tests
            variable_values = qaoa_result.x
            indexes_selected_tests = [index for index, value in enumerate(variable_values) if value == 1]
            print("Indexes of selected tests to convert. " + str(indexes_selected_tests))
            selected_tests = []
            for index in indexes_selected_tests:
                selected_tests.append(list(clusters_dictionary[sir_program].values())[cluster_dict_index][index])
            print("Selected tests: " + str(selected_tests))
            print("Experiment Number: " + str(i))
            cluster_dict_index += 1
            for selected_test in selected_tests:
                if selected_test not in final_selected_tests:
                    final_selected_tests.append(selected_test)
        i+=1
        #now we have to build the pareto front
        print("Final Selected Test Cases: " + str(final_selected_tests))
        print("Length of the final list of selected test cases: " + str(len(final_selected_tests)))
        start = time.time()
        pareto_front = build_pareto_front(sir_program, final_selected_tests)
        end = time.time()
        json_data["pareto_front_" + str(i)] = pareto_front
        pareto_front_building_time = (end - start) * 1000
        pareto_fronts_building_times.append(pareto_front_building_time)

    #compute the average time needed for the construction of a pareto frontier and run time
    mean_qpu_run_time = statistics.mean(qpu_run_times)
    mean_pareto_fronts_building_time = statistics.mean(pareto_fronts_building_times)
    json_data["mean_qpu_run_time(ms)"] = mean_qpu_run_time
    json_data["stdev_qpu_run_time(ms)"] = statistics.stdev(qpu_run_times)
    json_data["all_qpu_run_times(ms)"] = qpu_run_times
    json_data["mean_pareto_fronts_building_time(ms)"] = mean_pareto_fronts_building_time

    with open(file_path, "w") as file:
        json.dump(json_data, file)


## Fake Vigo Noise Simulator
The **Fake Vigo backend** in Qiskit Aer is a simulated version of IBM Quantum's **real 5-qubit Vigo quantum processor**. It allows users to test quantum circuits in a **realistic noisy environment** introducing **decoherence**.

In [ ]:
device_backend = FakeVigoV2()

algorithm_globals.random_seed = 10598

#I want to run the sampler 30 times to obtain different results for each sir program
for sir_program in sir_programs:
    qaoa_mes = QAOA(sampler=BackendSampler(backend=device_backend), optimizer=COBYLA(100), reps=sir_programs_rep_values[sir_program])
    qaoa = MinimumEigenOptimizer(qaoa_mes)  # using QAOA
    #the fronts will be saved into files
    print(sir_program)
    file_path = "results/selectqaoa/fake_vigo/" + sir_program + "-data.json"
    json_data = []
    response = None
    qpu_run_times = []
    pareto_fronts_building_times = []
    for i in range(experiments):
        final_selected_tests = []
        cluster_dict_index = 0
        for qubo in qubos_dictionary[sir_program]:
            print("Experiment Number: " + str(i))
            print("QUBO Problem: " + str(qubo) + "\nNumber: " + str(cluster_dict_index))
            print("Cluster's Test Cases: " +str(list(clusters_dictionary[sir_program].values())[cluster_dict_index]))
            #for each iteration get the result
            s = time.time()
            qaoa_result = qaoa.solve(qubo)
            print("RESULTS: " + str(qaoa_result))
            e = time.time()
            qpu_run_times.append((e - s) * 1000)
            #let's extract the selected tests
            variable_values = qaoa_result.x
            indexes_selected_tests = [index for index, value in enumerate(variable_values) if value == 1]
            print("Indexes of selected tests to convert. " + str(indexes_selected_tests))
            selected_tests = []
            for index in indexes_selected_tests:
                selected_tests.append(list(clusters_dictionary[sir_program].values())[cluster_dict_index][index])
            print("Selected tests: " + str(selected_tests))
            cluster_dict_index += 1
            for selected_test in selected_tests:
                if selected_test not in final_selected_tests:
                    final_selected_tests.append(selected_test)
        i+=1
        #now we have to build the pareto front
        print("Final Selected Test Cases: " + str(final_selected_tests))
        print(len(final_selected_tests))
        start = time.time()
        pareto_front = build_pareto_front(sir_program, final_selected_tests)
        end = time.time()
        json_data["pareto_front_" + str(i)] = pareto_front
        pareto_front_building_time = (end - start) * 1000
        pareto_fronts_building_times.append(pareto_front_building_time)

    #compute the average time needed for the construction of a pareto frontier and run time
    mean_qpu_run_time = statistics.mean(qpu_run_times)
    mean_pareto_fronts_building_time = statistics.mean(pareto_fronts_building_times)
    json_data["mean_qpu_run_time(ms)"] = mean_qpu_run_time
    json_data["stdev_qpu_run_time(ms)"] = statistics.stdev(qpu_run_times)
    json_data["all_qpu_run_times(ms)"] = qpu_run_times
    json_data["mean_pareto_fronts_building_time(ms)"] = mean_pareto_fronts_building_time

    with open(file_path, "w") as file:
        json.dump(json_data, file)


## Noise Mitigation in Quantum Computing

Quantum computers are inherently prone to errors due to decoherence, gate imperfections, and readout noise. These errors can significantly affect the accuracy of quantum algorithms, especially those relying on near-term noisy quantum hardware (NISQ devices). 

To combat these effects and extract more reliable results, **noise mitigation** techniques are employed. Unlike error correction, which requires additional qubits and complex encoding schemes, noise mitigation works at the software level and can be applied post-processing or during circuit execution.

### Zero-Noise Extrapolation (ZNE)

**Zero-Noise Extrapolation** is a leading noise mitigation strategy. It works by deliberately amplifying the noise in a controlled way and then using the results to extrapolate back to the zero-noise limit. Here's how it works:

1. **Run the same quantum circuit multiple times**, each time artificially increasing the noise level (e.g., by folding gates to increase circuit depth).
2. **Record the expectation values** (e.g., energy or measurement outcomes) for each noise level.
3. **Fit a curve** (e.g., linear, quadratic) through these noisy points.
4. **Extrapolate back** to the zero-noise value (i.e., what the result would be if there were no noise).



In [ ]:
def expectation_from_distribution(distribution, operator):
    expectation = 0.0
    for pauli, coeff in zip(operator.paulis, operator.coeffs):
        pauli_str = pauli.to_label()
        if any(p in pauli_str for p in "XY"):
            continue
        for bitstring, prob in distribution.items():
            if len(pauli_str) < len(bitstring):
                pauli_str = pauli_str.rjust(len(bitstring), "I")
            parity = 1
            for i in range(len(bitstring)):
                p = pauli_str[-(i + 1)]
                if p == 'Z':
                    parity *= 1 if bitstring[-(i + 1)] == '0' else -1
            expectation += coeff.real * prob * parity
    return expectation.real

# Setup backend and seed
device_backend = FakeVigoV2()
algorithm_globals.random_seed = 10598

# Setup ZNE
scale_factors = [1.0, 3.0, 5.0]
factory = RichardsonFactory(scale_factors=scale_factors)
scale_noise = fold_global

# Main loop
for sir_program in sir_programs:
    print(f"Running program: {sir_program}")
    file_path = f"results/selectqaoa/fake_vigo_zne/{sir_program}-data.json"
    json_data = {}
    qpu_run_times = []
    pareto_fronts_building_times = []

    for i in range(experiments):
        final_selected_tests = []
        cluster_dict_index = 0

        for qubo in qubos_dictionary[sir_program]:
            print(f"Experiment {i}, QUBO {cluster_dict_index}")
            cluster_tests = list(clusters_dictionary[sir_program].values())[cluster_dict_index]
            print("Cluster's Test Cases:", cluster_tests)

            # Convert QUBO to Ising
            qp2qubo = QuadraticProgramToQubo()
            qubo_problem = qp2qubo.convert(qubo)
            hamiltonian, _ = qubo_problem.to_ising()
            reps = sir_programs_rep_values[sir_program]
            ansatz = QAOAAnsatz(hamiltonian, reps=reps)

            def zne_expectation(params):
                circuit = ansatz.assign_parameters(params)
                circuit.measure_all()
                def run_expectation(circ):
                    sampler = BackendSampler(backend=device_backend)
                    result = sampler.run(circ).result()
                    dist = result.quasi_dists[0].binary_probabilities()
                    return expectation_from_distribution(dist, hamiltonian)
                return zne.execute_with_zne(circuit, run_expectation, scale_noise=scale_noise, factory=factory)

            initial_params = np.array([0.5] * ansatz.num_parameters)
            result = minimize(zne_expectation, initial_params, method="COBYLA")
            optimal_params = result.x

            final_circuit = ansatz.assign_parameters(optimal_params)
            final_circuit.measure_all()
            sampler = BackendSampler(backend=device_backend)

            s = time.time()
            final_result = sampler.run(final_circuit).result()
            e = time.time()
            dist = final_result.quasi_dists[0].binary_probabilities()
            best_bitstring = max(dist.items(), key=lambda x: x[1])[0]
            qpu_run_times.append((e - s) * 1000)

            selected_tests = [cluster_tests[j] for j, bit in enumerate(best_bitstring[::-1]) if bit == '1']
            print(f"Selected tests: {selected_tests}")

            for test in selected_tests:
                if test not in final_selected_tests:
                    final_selected_tests.append(test)

            cluster_dict_index += 1

        print("Final selected tests:", final_selected_tests)
        print("Number of tests selected:", len(final_selected_tests))

        start = time.time()
        pareto_front = build_pareto_front(sir_program, final_selected_tests)
        end = time.time()
        pareto_fronts_building_times.append((end - start) * 1000)
        json_data[f"pareto_front_{i}"] = pareto_front

    json_data["mean_qpu_run_time(ms)"] = statistics.mean(qpu_run_times)
    json_data["stdev_qpu_run_time(ms)"] = statistics.stdev(qpu_run_times)
    json_data["all_qpu_run_times(ms)"] = qpu_run_times
    json_data["mean_pareto_fronts_building_time(ms)"] = statistics.mean(pareto_fronts_building_times)

    os.makedirs(os.path.dirname(file_path), exist_ok=True)
    with open(file_path, "w") as f:
        json.dump(json_data, f, indent=4)

    print(f"Results saved to {file_path}")


## Depolarizing Error Simulator
The **depolarizing noise model** is a common quantum noise model that simulates the effect of random errors occurring in quantum computations. In a **depolarizing channel**, a qubit undergoes a transformation where it has a probability \( p \) of being replaced by a completely mixed state, losing all its quantum coherence.

- **1% noise** simulates near-term quantum hardware with optimized error rates.
- **2% noise** represents moderate errors in typical superconducting quantum processors.
- **5% noise** helps study worst-case scenarios and assess robustness of quantum algorithms.
- Comparing different noise levels allows us to evaluate how quantum circuits **degrade** under varying degrees of decoherence and to **develop error mitigation techniques**.


In [ ]:
noise_dep = NoiseModel()

# Define a 1% depolarizing error for single-qubit and two-qubit gates
depolarizing_error_1q = depolarizing_error(0.01, 1)  # 1% depolarizing error for single-qubit gates
depolarizing_error_2q = depolarizing_error(0.01, 2)  # 1% depolarizing error for two-qubit gates

# Apply depolarizing noise to single-qubit gates
noise_dep.add_all_qubit_quantum_error(depolarizing_error_1q, ['rx', 'rz', 'sx', 'x', 'h'])

# Apply depolarizing noise to two-qubit gates
noise_dep.add_all_qubit_quantum_error(depolarizing_error_2q, ['cx', 'cz', 'swap'])

sim_dep = AerSimulator(noise_model=noise_dep)

algorithm_globals.random_seed = 10598

#I want to run the sampler 30 times to obtain different results for each sir program
for sir_program in sir_programs:
    qaoa_mes = QAOA(sampler=BackendSampler(backend=sim_dep), optimizer=COBYLA(100), reps=sir_programs_rep_values[sir_program])
    qaoa = MinimumEigenOptimizer(qaoa_mes)  # using QAOA
    #the fronts will be saved into files
    print(sir_program)
    file_path = "results/selectqaoa/depolarizing_sim/01/" + sir_program + "-data.json"
    json_data = []
    response = None
    qpu_run_times = []
    pareto_fronts_building_times = []
    for i in range(experiments):
        final_selected_tests = []
        cluster_dict_index = 0
        for qubo in qubos_dictionary[sir_program]:
            print("Experiment Number: " + str(i))
            print("QUBO Problem: " + str(qubo) + "\nNumber: " + str(cluster_dict_index))
            print("Cluster's Test Cases: " +str(list(clusters_dictionary[sir_program].values())[cluster_dict_index]))
            #for each iteration get the result
            s = time.time()
            qaoa_result = qaoa.solve(qubo)
            print("RESULTS: " + str(qaoa_result))
            e = time.time()
            qpu_run_times.append((e - s) * 1000)
            #let's extract the selected tests
            variable_values = qaoa_result.x
            indexes_selected_tests = [index for index, value in enumerate(variable_values) if value == 1]
            print("Indexes of selected tests to convert. " + str(indexes_selected_tests))
            selected_tests = []
            for index in indexes_selected_tests:
                selected_tests.append(list(clusters_dictionary[sir_program].values())[cluster_dict_index][index])
            print("Selected tests: " + str(selected_tests))
            cluster_dict_index += 1
            for selected_test in selected_tests:
                if selected_test not in final_selected_tests:
                    final_selected_tests.append(selected_test)
        i+=1
        #now we have to build the pareto front
        print("Final Selected Test Cases: " + str(final_selected_tests))
        print(len(final_selected_tests))
        start = time.time()
        pareto_front = build_pareto_front(sir_program, final_selected_tests)
        end = time.time()
        json_data["pareto_front_" + str(i)] = pareto_front
        pareto_front_building_time = (end - start) * 1000
        pareto_fronts_building_times.append(pareto_front_building_time)

    #compute the average time needed for the construction of a pareto frontier and run time
    mean_qpu_run_time = statistics.mean(qpu_run_times)
    mean_pareto_fronts_building_time = statistics.mean(pareto_fronts_building_times)
    json_data["mean_qpu_run_time(ms)"] = mean_qpu_run_time
    json_data["stdev_qpu_run_time(ms)"] = statistics.stdev(qpu_run_times)
    json_data["all_qpu_run_times(ms)"] = qpu_run_times
    json_data["mean_pareto_fronts_building_time(ms)"] = mean_pareto_fronts_building_time

    with open(file_path, "w") as file:
        json.dump(json_data, file)


In [ ]:
noise_dep = NoiseModel()

# Define a 2% depolarizing error for single-qubit and two-qubit gates
depolarizing_error_1q = depolarizing_error(0.02, 1)  # 2% depolarizing error for single-qubit gates
depolarizing_error_2q = depolarizing_error(0.02, 2)  # 2% depolarizing error for two-qubit gates

# Apply depolarizing noise to single-qubit gates
noise_dep.add_all_qubit_quantum_error(depolarizing_error_1q, ['rx', 'rz', 'sx', 'x', 'h'])

# Apply depolarizing noise to two-qubit gates
noise_dep.add_all_qubit_quantum_error(depolarizing_error_2q, ['cx', 'cz', 'swap'])

sim_dep = AerSimulator(noise_model=noise_dep)

algorithm_globals.random_seed = 10598

#I want to run the sampler 30 times to obtain different results for each sir program
for sir_program in sir_programs:
    qaoa_mes = QAOA(sampler=BackendSampler(backend=sim_dep), optimizer=COBYLA(100), reps=sir_programs_rep_values[sir_program])
    qaoa = MinimumEigenOptimizer(qaoa_mes)  # using QAOA
    #the fronts will be saved into files
    print(sir_program)
    file_path = "results/selectqaoa/depolarizing_sim/02/" + sir_program + "-data.json"
    json_data = []
    response = None
    qpu_run_times = []
    pareto_fronts_building_times = []
    for i in range(experiments):
        final_selected_tests = []
        cluster_dict_index = 0
        for qubo in qubos_dictionary[sir_program]:
            print("Experiment Number: " + str(i))
            print("QUBO Problem: " + str(qubo) + "\nNumber: " + str(cluster_dict_index))
            print("Cluster's Test Cases: " +str(list(clusters_dictionary[sir_program].values())[cluster_dict_index]))
            #for each iteration get the result
            s = time.time()
            qaoa_result = qaoa.solve(qubo)
            print("RESULTS: " + str(qaoa_result))
            e = time.time()
            qpu_run_times.append((e - s) * 1000)
            #let's extract the selected tests
            variable_values = qaoa_result.x
            indexes_selected_tests = [index for index, value in enumerate(variable_values) if value == 1]
            print("Indexes of selected tests to convert. " + str(indexes_selected_tests))
            selected_tests = []
            for index in indexes_selected_tests:
                selected_tests.append(list(clusters_dictionary[sir_program].values())[cluster_dict_index][index])
            print("Selected tests: " + str(selected_tests))
            cluster_dict_index += 1
            for selected_test in selected_tests:
                if selected_test not in final_selected_tests:
                    final_selected_tests.append(selected_test)
        i+=1
        #now we have to build the pareto front
        print("Final Selected Test Cases: " + str(final_selected_tests))
        print(len(final_selected_tests))
        start = time.time()
        pareto_front = build_pareto_front(sir_program, final_selected_tests)
        end = time.time()
        json_data["pareto_front_" + str(i)] = pareto_front
        pareto_front_building_time = (end - start) * 1000
        pareto_fronts_building_times.append(pareto_front_building_time)

    #compute the average time needed for the construction of a pareto frontier and run time
    mean_qpu_run_time = statistics.mean(qpu_run_times)
    mean_pareto_fronts_building_time = statistics.mean(pareto_fronts_building_times)
    json_data["mean_qpu_run_time(ms)"] = mean_qpu_run_time
    json_data["stdev_qpu_run_time(ms)"] = statistics.stdev(qpu_run_times)
    json_data["all_qpu_run_times(ms)"] = qpu_run_times
    json_data["mean_pareto_fronts_building_time(ms)"] = mean_pareto_fronts_building_time

    with open(file_path, "w") as file:
        json.dump(json_data, file)


In [ ]:
noise_dep = NoiseModel()

# Define a 5% depolarizing error for single-qubit and two-qubit gates
depolarizing_error_1q = depolarizing_error(0.05, 1)  # 5% depolarizing error for single-qubit gates
depolarizing_error_2q = depolarizing_error(0.05, 2)  # 5% depolarizing error for two-qubit gates

# Apply depolarizing noise to single-qubit gates
noise_dep.add_all_qubit_quantum_error(depolarizing_error_1q, ['rx', 'rz', 'sx', 'x', 'h'])

# Apply depolarizing noise to two-qubit gates
noise_dep.add_all_qubit_quantum_error(depolarizing_error_2q, ['cx', 'cz', 'swap'])

sim_dep = AerSimulator(noise_model=noise_dep)

algorithm_globals.random_seed = 10598

#I want to run the sampler 30 times to obtain different results for each sir program
for sir_program in sir_programs:
    qaoa_mes = QAOA(sampler=BackendSampler(backend=sim_dep), optimizer=COBYLA(100), reps=sir_programs_rep_values[sir_program])
    qaoa = MinimumEigenOptimizer(qaoa_mes)  # using QAOA
    #the fronts will be saved into files
    print(sir_program)
    file_path = "results/selectqaoa/depolarizing_sim/05/" + sir_program + "-data.json"
    json_data = []
    response = None
    qpu_run_times = []
    pareto_fronts_building_times = []
    for i in range(experiments):
        final_selected_tests = []
        cluster_dict_index = 0
        for qubo in qubos_dictionary[sir_program]:
            print("Experiment Number: " + str(i))
            print("QUBO Problem: " + str(qubo) + "\nNumber: " + str(cluster_dict_index))
            print("Cluster's Test Cases: " +str(list(clusters_dictionary[sir_program].values())[cluster_dict_index]))
            #for each iteration get the result
            s = time.time()
            qaoa_result = qaoa.solve(qubo)
            print("RESULTS: " + str(qaoa_result))
            e = time.time()
            qpu_run_times.append((e - s) * 1000)
            #let's extract the selected tests
            variable_values = qaoa_result.x
            indexes_selected_tests = [index for index, value in enumerate(variable_values) if value == 1]
            print("Indexes of selected tests to convert. " + str(indexes_selected_tests))
            selected_tests = []
            for index in indexes_selected_tests:
                selected_tests.append(list(clusters_dictionary[sir_program].values())[cluster_dict_index][index])
            print("Selected tests: " + str(selected_tests))
            cluster_dict_index += 1
            for selected_test in selected_tests:
                if selected_test not in final_selected_tests:
                    final_selected_tests.append(selected_test)
        i+=1
        #now we have to build the pareto front
        print("Final Selected Test Cases: " + str(final_selected_tests))
        print(len(final_selected_tests))
        start = time.time()
        pareto_front = build_pareto_front(sir_program, final_selected_tests)
        end = time.time()
        json_data["pareto_front_" + str(i)] = pareto_front
        pareto_front_building_time = (end - start) * 1000
        pareto_fronts_building_times.append(pareto_front_building_time)

    #compute the average time needed for the construction of a pareto frontier and run time
    mean_qpu_run_time = statistics.mean(qpu_run_times)
    mean_pareto_fronts_building_time = statistics.mean(pareto_fronts_building_times)
    json_data["mean_qpu_run_time(ms)"] = mean_qpu_run_time
    json_data["stdev_qpu_run_time(ms)"] = statistics.stdev(qpu_run_times)
    json_data["all_qpu_run_times(ms)"] = qpu_run_times
    json_data["mean_pareto_fronts_building_time(ms)"] = mean_pareto_fronts_building_time

    with open(file_path, "w") as file:
        json.dump(json_data, file)


## Effectiveness Empirical Evaluations
The four algorithms were evaluated in terms of their **effectiveness**. In particular, we built a **reference Pareto front** made by all the **non-dominated solutions** found by all the algorithms during all ten runs. 

Let $P = \{ P_1, ..., P_l \}$ be the set of \( $l$ \) different Pareto fronts obtained after all the experiment runs by all the evaluated algorithms. The **reference Pareto front** $P_{\text{ref}}$ is defined as follows:


$P_{\text{ref}} \subseteq \bigcup_{i=1}^{l} P_i$


where $\forall p \in P_{\text{ref}}, \nexists q \in P_{\text{ref}}: q > p$. 

Given the reference front, we computed as an **effectiveness metric** the **number of non-dominated solutions**, i.e., the number of non-dominated solutions found by an algorithm selected for the final reference front.

To ensure the empirical reliability of the results, we used the **Mann-Whitney U test** and the **Vargha-Delaney effect size** $\hat{A}_{12}$. The null hypothesis, i.e., there is **no statistically significant difference** between the effectiveness of the algorithms, is rejected if p-values < 0.05.


### The Dunn's Test

The **Kruskal-Wallis H-test** is a non-parametric statistical test used to determine whether there are significant differences between three or more independent groups. However, while the test can indicate that at least one group differs, it does not specify which groups are different from each other.

To address this, we apply **Dunn's Test**, a post-hoc pairwise comparison method that helps identify which specific groups differ. This test compares all possible pairs of groups to determine which ones have statistically significant differences. Since multiple comparisons increase the chance of false positives, Dunn's test is often used with a correction method such as Bonferroni or Holm to adjust p-values. If the adjusted p-value is below the significance threshold (typically 0.05), we conclude that the two compared groups are significantly different.



In [ ]:
#the other sizes are uniform
#this static information can be easily obtained from the pareto .json files

qtcs_flex_pareto_sizes = [187,187,187,187,187,187,187,187,187,187]
divga_flex_pareto_sizes = [140,140,140,140,140,140,140,140,140,140]
qaoa_statevector_flex_pareto_sizes = [447,434,413,409,434,455,446,410,416,427]
qaoa_aer_flex_pareto_sizes = [517,515,518,523,516,518,505,514,518,526]
qaoa_fakevigo_flex_pareto_sizes = [540,540,540,540,540,540,540,539,539,539]
qaoa_noise1_flex_pareto_sizes = [539,540,540,539,540,540,539,539,539,539]
qaoa_noise2_flex_pareto_sizes = [540,539,540,539,540,540,539,540,539,540]
qaoa_noise5_flex_pareto_sizes = [539,539,540,540,540,539,540,540,540,539]
greedy_flex_size = 567

qtcs_grep_pareto_sizes = [229,229,230,229,230,229,230.229,230,230]
divga_grep_pareto_sizes = [70,70,70,70,70,70,70,70,70,70]
qaoa_statevector_grep_pareto_sizes = [485,507,504,505,489,492,489,493,489,470]
qaoa_aer_grep_pareto_sizes = [343,345,343,341,346,345,345,346,340,348]
qaoa_fakevigo_grep_pareto_sizes = [321,321,321,321,321,321,321,321,321,321]
qaoa_noise1_grep_pareto_sizes = [321,321,321,321,321,321,321,321,321,321]
qaoa_noise2_grep_pareto_sizes = [321,321,321,321,321,321,321,321,321,321]
qaoa_noise5_grep_pareto_sizes = [321,321,321,321,321,321,321,321,321,321]
greedy_grep_size = 802

qtcs_gzip_pareto_sizes = [87,86,85,86,87,87,86,87,85,87]
divga_gzip_pareto_sizes = [105,105,105,105,105,105,105,105,105,105]
qaoa_statevector_gzip_pareto_sizes = [137,143,146,144,164,143,157,144,141,169]
qaoa_aer_gzip_pareto_sizes = [147,146,145,146,151,155,155,151,154,156]
qaoa_fakevigo_gzip_pareto_sizes = [168,168,166,169,168,167,167,168,168,167]
qaoa_noise1_gzip_pareto_sizes = [167,167,167,166,168,167,166,166,168,166]
qaoa_noise2_gzip_pareto_sizes = [166,166,168,167,166,166,166,167,167,166]
qaoa_noise5_gzip_pareto_sizes = [169,167,167,169,168,168,168,165,167,167]
greedy_gzip_size = 199

qtcs_sed_pareto_sizes = [131,131,131,131,131,131,131,131,131,131]
divga_sed_pareto_sizes = [105,62,105,105,102,105,105,97,105,105]
qaoa_statevector_sed_pareto_sizes = [249,241,252,226,245,231,248,241,245,237]
qaoa_aer_sed_pareto_sizes = [260,251,253,263,252,263,260,256,250,253]
qaoa_fakevigo_sed_pareto_sizes = [235,235,236,235,236,236,235,236,236,236]
qaoa_noise1_sed_pareto_sizes = [236,235,235,235,236,236,236,235,236,235]
qaoa_noise2_sed_pareto_sizes = [235,235,236,235,235,236,236,236,236,236]
qaoa_noise5_sed_pareto_sizes = [236,236,235,236,236,235,235,236,236,235]
greedy_sed_size = 356

In [ ]:
# Unpack the tuples for each list
def unpack_tuples(data):
    """Unpack the data"""
    return zip(*data)

json_data = {"flex":{"statevector_sim":[],"aer_sim":[],"fake_vigo":[],"depolarizing_sim/01":[],"depolarizing_sim/02":[],"depolarizing_sim/05":[]},"grep":{"statevector_sim":[],"aer_sim":[],"fake_vigo":[],"depolarizing_sim/01":[],"depolarizing_sim/02":[],"depolarizing_sim/05":[]},"gzip":{"statevector_sim":[],"aer_sim":[],"fake_vigo":[],"depolarizing_sim/01":[],"depolarizing_sim/02":[],"depolarizing_sim/05":[]},"sed":{"statevector_sim":[],"aer_sim":[],"fake_vigo":[],"depolarizing_sim/01":[],"depolarizing_sim/02":[],"depolarizing_sim/05":[]}}
normal_dist = True
for sir_program in sir_programs:
    print("================="+sir_program+"=================")
    with open('./results/add-greedy/'+sir_program+'_data.json', 'r') as file:
        pareto_fronts_json = json.load(file)
    
    greedy_pareto_front = pareto_fronts_json['pareto_front']
    
    #each solution of the pareto front is a subset of the test suite
    greedy_pareto_vectors = []
    for front_solution in greedy_pareto_front:
        #compute the fitness scores of each single solution
        greedy_pareto_vectors.append((total_cost(sir_program,front_solution),total_coverage(sir_program,front_solution),total_faults(sir_program,front_solution)))
        #here we make the same but for each of the fronts built by SelectQA and DIV-GA

    #the following 2 lists will contain much tuples as solutions in each of the 10 pareto fronts
    #each tuple represent the fitness value for each solution
    qtcs_pareto_vectors = []
    divga_pareto_vectors = []
    qaoa_statevector_pareto_vectors = []
    qaoa_aer_pareto_vectors = []
    qaoa_fakevigo_pareto_vectors = []
    qaoa_noise1_pareto_vectors = []
    qaoa_noise2_pareto_vectors = []
    qaoa_noise5_pareto_vectors = []
    
    for index in range(0,10):
        # Load the JSON file
        with open('results/selectqa/'+sir_program+'-data.json', 'r') as file:
            pareto_fronts_json = json.load(file)
        
        qtcs_pareto_front = pareto_fronts_json['pareto_front_'+str(index)]
        #a single solution is a subset of the initial test suite
        for front_solution in qtcs_pareto_front:
            qtcs_pareto_vectors.append((total_cost(sir_program,front_solution),total_coverage(sir_program,front_solution),total_faults(sir_program,front_solution)))
        #print(len(qtcs_pareto_vectors))
        
        with open('./results/divga/'+sir_program+'_pareto_fronts_divga.json', 'r') as file:
            pareto_fronts_json = json.load(file)
        
        divga_pareto_front = pareto_fronts_json[sir_program+'_pareto_front_'+str(index)]
        for front_solution in divga_pareto_front:
            divga_pareto_vectors.append((total_cost(sir_program,front_solution),total_coverage(sir_program,front_solution),total_faults(sir_program,front_solution)))
        #print(len(divga_pareto_vectors))
        
        #the .json file extracted for the statevector simulator is the copy of the optimal one (between 1,2,4,8,16)
        with open('results/selectqaoa/statevector_sim/'+sir_program+'-data.json', 'r') as file:
            pareto_fronts_json = json.load(file)
        
        qaoa_pareto_front = pareto_fronts_json['pareto_front_'+str(index+1)]
        #a single solution is a subset of the initial test suite
        for front_solution in qaoa_pareto_front:
            qaoa_statevector_pareto_vectors.append((total_cost(sir_program,front_solution),total_coverage(sir_program,front_solution),total_faults(sir_program,front_solution)))
        #print(len(qaoa_statevector_pareto_vectors))
        
        with open('results/selectqaoa/aer_sim/'+sir_program+'-data.json', 'r') as file:
            pareto_fronts_json = json.load(file)
        
        qaoa_pareto_front = pareto_fronts_json['pareto_front_'+str(index+1)]
        #a single solution is a subset of the initial test suite
        for front_solution in qaoa_pareto_front:
            qaoa_aer_pareto_vectors.append((total_cost(sir_program,front_solution),total_coverage(sir_program,front_solution),total_faults(sir_program,front_solution)))
        #print(len(qaoa_aer_pareto_vectors))
        
        with open('results/selectqaoa/fake_vigo/'+sir_program+'-data.json', 'r') as file:
            pareto_fronts_json = json.load(file)
        
        qaoa_pareto_front = pareto_fronts_json['pareto_front_'+str(index+1)]
        #a single solution is a subset of the initial test suite
        for front_solution in qaoa_pareto_front:
            qaoa_fakevigo_pareto_vectors.append((total_cost(sir_program,front_solution),total_coverage(sir_program,front_solution),total_faults(sir_program,front_solution)))
        #print(len(qaoa_fakevigo_pareto_vectors))
        
        with open('results/selectqaoa/depolarizing_sim/01/'+sir_program+'-data.json', 'r') as file:
            pareto_fronts_json = json.load(file)
        
        qaoa_pareto_front = pareto_fronts_json['pareto_front_'+str(index+1)]
        #a single solution is a subset of the initial test suite
        for front_solution in qaoa_pareto_front:
            qaoa_noise1_pareto_vectors.append((total_cost(sir_program,front_solution),total_coverage(sir_program,front_solution),total_faults(sir_program,front_solution)))
        #print(len(qaoa_noise1_pareto_vectors))
        
        with open('results/selectqaoa/depolarizing_sim/02/'+sir_program+'-data.json', 'r') as file:
            pareto_fronts_json = json.load(file)
        
        qaoa_pareto_front = pareto_fronts_json['pareto_front_'+str(index+1)]
        #a single solution is a subset of the initial test suite
        for front_solution in qaoa_pareto_front:
            qaoa_noise2_pareto_vectors.append((total_cost(sir_program,front_solution),total_coverage(sir_program,front_solution),total_faults(sir_program,front_solution)))
        #print(len(qaoa_noise2_pareto_vectors))
        
        with open('results/selectqaoa/depolarizing_sim/05/'+sir_program+'-data.json', 'r') as file:
            pareto_fronts_json = json.load(file)
        
        qaoa_pareto_front = pareto_fronts_json['pareto_front_'+str(index+1)]
        #a single solution is a subset of the initial test suite
        for front_solution in qaoa_pareto_front:
            qaoa_noise5_pareto_vectors.append((total_cost(sir_program,front_solution),total_coverage(sir_program,front_solution),total_faults(sir_program,front_solution)))
        #print(len(qaoa_noise5_pareto_vectors))
        
    qaoa_pareto_vectors = [qaoa_statevector_pareto_vectors,qaoa_aer_pareto_vectors,qaoa_fakevigo_pareto_vectors,qaoa_noise1_pareto_vectors,qaoa_noise2_pareto_vectors,qaoa_noise5_pareto_vectors]
    qaoa_configs = ["statevector_sim","aer_sim","fake_vigo","depolarizing_sim/01","depolarizing_sim/02","depolarizing_sim/05"]
    
    for i in range(len(qaoa_pareto_vectors)):
        
        total_fronts = [qtcs_pareto_vectors,divga_pareto_vectors,greedy_pareto_vectors,qaoa_pareto_vectors[i]]
        reference_pareto = []
        portions = [0,0,0,0]
        
        # get the reference frontier
        for index, front1 in enumerate(total_fronts):
            for front_solution1 in front1:
                is_dominated = 0
                other_fronts = total_fronts[:index] + total_fronts[index+1:]
                for front2 in other_fronts:
                    for front_solution2 in front2:
                        if pareto_dominance(front_solution1,front_solution2):
                            is_dominated = 1
                            break
                    if is_dominated:
                        break
                if not is_dominated:
                    reference_pareto.append(front_solution1)
                    portions[index] = portions[index] + 1
        
        #print(portions)
        
        qtcs_non_dominated_values = []
        divga_non_dominated_values = []
        qaoa_non_dominated_values = []
        greedy_non_dominated = 0
        
        for index in range(0,10):
            qtcs_non_dominated = 0
            divga_non_dominated = 0
            qaoa_non_dominated = 0
            
            # Load the JSON file
            with open('results/selectqa/'+sir_program+'-data.json', 'r') as file:
                pareto_fronts_json = json.load(file)
            
            qtcs_pareto_front = pareto_fronts_json['pareto_front_'+str(index)]
            for front_solution in qtcs_pareto_front:
                if (total_cost(sir_program,front_solution),total_coverage(sir_program,front_solution),total_faults(sir_program,front_solution)) in reference_pareto:
                    qtcs_non_dominated += 1
            
            qtcs_non_dominated_values.append(qtcs_non_dominated)
            #print(qtcs_non_dominated)
            
            with open('./results/divga/'+sir_program+'_pareto_fronts_divga.json', 'r') as file:
                pareto_fronts_json = json.load(file)
            
            divga_pareto_front = pareto_fronts_json[sir_program+'_pareto_front_'+str(index)]
            for front_solution in divga_pareto_front:
                if (total_cost(sir_program,front_solution),total_coverage(sir_program,front_solution),total_faults(sir_program,front_solution)) in reference_pareto:
                    divga_non_dominated += 1
                    
            divga_non_dominated_values.append(divga_non_dominated)
            #print(divga_non_dominated)
            
            with open('results/selectqaoa/'+qaoa_configs[i]+'/'+sir_program+'-data.json', 'r') as file:
                pareto_fronts_json = json.load(file)
            
            qaoa_pareto_front = pareto_fronts_json['pareto_front_'+str(index+1)]
            for front_solution in qaoa_pareto_front:
                if (total_cost(sir_program,front_solution),total_coverage(sir_program,front_solution),total_faults(sir_program,front_solution)) in reference_pareto:
                    qaoa_non_dominated += 1
            
            qaoa_non_dominated_values.append(qaoa_non_dominated)
            #print(qaoa_statevector_non_dominated)
            
        for front_solution in greedy_pareto_front:
            if (total_cost(sir_program,front_solution),total_coverage(sir_program,front_solution),total_faults(sir_program,front_solution)) in reference_pareto:
                greedy_non_dominated += 1
        #print(greedy_non_dominated)
        
        print("================="+sir_program+"=================")
        print("QTCS/DIVGA/ADD GREEDY")
        print("================="+str(qaoa_configs[i])+"=================")
        print("SelectQA Non Dominated Values")
        print(qtcs_non_dominated_values)
        print("SelectQA Non Dominated Mean")
        print(statistics.mean(qtcs_non_dominated_values))
        print("SelectQA Non Dominated StDev")
        print(statistics.stdev(qtcs_non_dominated_values))
        json_data[sir_program][qaoa_configs[i]]["selectqa"] = (qtcs_non_dominated_values,statistics.mean(qtcs_non_dominated_values),statistics.stdev(qtcs_non_dominated_values))
        
        print("DIV-GA Non Dominated Values")
        print(divga_non_dominated_values)
        print("DIV-GA Non Dominated Mean")
        print(statistics.mean(divga_non_dominated_values))
        print("DIV-GA Non Dominated StDev")
        print(statistics.stdev(divga_non_dominated_values))
        json_data[sir_program][qaoa_configs[i]]["divga"] = (divga_non_dominated_values,statistics.mean(divga_non_dominated_values),statistics.stdev(divga_non_dominated_values))
        
        print("SelectQAOA Non Dominated Values ("+str(qaoa_configs[i])+")")
        print(qaoa_non_dominated_values)
        print("SelectQAOA Non Dominated Mean ("+str(qaoa_configs[i])+")")
        print(statistics.mean(qaoa_non_dominated_values))
        print("SelectQAOA Non Dominated StDev ("+str(qaoa_configs[i])+")")
        print(statistics.stdev(qaoa_non_dominated_values))
        json_data[sir_program][qaoa_configs[i]]["qaoa"] = (qaoa_non_dominated_values,statistics.mean(qaoa_non_dominated_values),statistics.stdev(qaoa_non_dominated_values))
        
        print("Greedy Non Dominated Values")
        print(greedy_non_dominated)
        json_data[sir_program][qaoa_configs[i]]["add_greedy"] = (greedy_non_dominated,greedy_non_dominated,0)
        
        print(json_data)
        
        if normal_dist:
            lists = [qtcs_non_dominated_values,divga_non_dominated_values,qaoa_non_dominated_values]
    
            # we are interested in knowing if the obtained sequences are normally distributed to decide what statistical test we should apply
            for i, list in enumerate(lists, start=1):
                stat, p_value = shapiro(list)
                #print(f"List {i}: Statistic = {stat:.5f}, P-value = {p_value:.5f}")
                
                # check if the sequence is normally distributed (using 0.05 as threshold)
                if p_value <= 0.05:
                    #print(f"List {i} NOT seems to be normally distributed (p-value = {p_value:.5f})")
                    normal_dist = False
        
        # print the plots
        best_pareto_vectors = None
        means = [statistics.mean(qtcs_non_dominated_values),statistics.mean(divga_non_dominated_values),statistics.mean(qaoa_non_dominated_values),greedy_non_dominated]
        best_mean_index = means.index(max(means))
        if best_mean_index == 0:
            best_pareto_vectors = qtcs_pareto_vectors
            msg = "SelectQA"
        elif best_mean_index == 1:
            best_pareto_vectors = divga_pareto_vectors
            msg = "DIV-GA"
        elif best_mean_index == 2:
            if i == 0:
                best_pareto_vectors = qaoa_statevector_pareto_vectors
                msg = "Statevector SelectQAOA"
            elif i == 1:
                best_pareto_vectors = qaoa_aer_pareto_vectors
                msg = "Aer Simulator SelectQAOA"
            elif i == 2:
                best_pareto_vectors = qaoa_fakevigo_pareto_vectors
                msg = "FakeVigo SelectQAOA"
            elif i == 3:
                best_pareto_vectors = qaoa_noise1_pareto_vectors
                msg = "Dep Err 1 SelectQAOA"
            elif i == 4:
                best_pareto_vectors = qaoa_noise2_pareto_vectors
                msg = "Dep Err 2 SelectQAOA"
            elif i == 5:
                best_pareto_vectors = qaoa_noise5_pareto_vectors
                msg = "Dep Err 5 SelectQAOA"
        else:
            best_pareto_vectors = greedy_pareto_vectors
            msg = "Additional Greedy" 
        x1, y1, z1 = unpack_tuples(best_pareto_vectors)
        x2, y2, z2 = unpack_tuples(reference_pareto)
        
        # Create 3D scatter plots
        fig = plt.figure(figsize=(18,6))
        
        ax = fig.add_subplot(133, projection='3d')
        ax.scatter(x1, y1, z1, c='m', marker='s', s=100, label=msg)
        ax.scatter(x2, y2, z2, c='b', marker='o', s=50, label='Reference Pareto')
        ax.set_xlabel('Cost')
        ax.set_ylabel('Statement Coverage')
        ax.set_zlabel('Past Faults',labelpad=0.2)
        ax.set_title(msg + ' vs Reference Pareto')
        ax.legend()

        # Show the plots
        plt.tight_layout()
        plt.show()

if normal_dist:
    print("Values all normally distributed")
else:
    print("Values not normally distributed")

with open("multi_obj_frontiers_eval.json", "w") as file:
    json.dump(json_data, file)

## Efficiency Empirical Evaluations
Regarding the efficiency, we analyzed the algorithms’ **total execution times**. The total run time of **SelectQAOA** was computed by summing the **decomposition** and **execution times** of its run on a **Barbora (IT4Innovations supercomputer)** featuring 8 cores of the **Intel Xeon Gold 6240 CPU** with **2.60 GHz**. 

For **SelectQA**, executed on the **D-Wave hybrid_binary_quadratic_model_version2**, we considered the **D-Wave API "total run time"** metric. 

The other algorithms' execution times were computed on an **Apple MacBook Air** featuring an **M1 chip** and **16GB of RAM**.

Finally, to ensure the empirical reliability of the results, we used the **Mann-Whitney U test** and the **Vargha-Delaney effect size** $\hat{A}_{12}$.

In [ ]:
versions = ["statevector_sim", "aer_sim", "fake_vigo", "depolarizing_sim/01", "depolarizing_sim/02", "depolarizing_sim/05"]

def is_not_normal(data):
    if len(data) < 3:
        return False  # troppo pochi dati per testare
    _, p_value = shapiro(data)
    return p_value < 0.05

def read_json_field(path, field):
    try:
        with open(path, 'r') as f:
            data = json.load(f)
        return data.get(field, [])
    except FileNotFoundError:
        return []

for sir_program in sir_programs:
    not_normal = False

    # QTCs
    qtcs_path = f"results/selectqa/{sir_program}-data.json"
    qtcs_times = read_json_field(qtcs_path, "run_times(ms)")
    if is_not_normal(qtcs_times):
        not_normal = True

    # Add-greedy
    add_greedy_path = f"results/add-greedy/{sir_program}_data.json"
    add_greedy_time = read_json_field(add_greedy_path, "resolution_time(ms)")
    values = add_greedy_time if isinstance(add_greedy_time, list) else [add_greedy_time]
    if is_not_normal(values):
        not_normal = True

    # DivGA
    divga_path = f"results/divga/{sir_program}_pareto_fronts_divga.json"
    divga_times = read_json_field(divga_path, "execution_times")
    if is_not_normal(divga_times):
        not_normal = True

    # QAOA
    for version in versions:
        qaoa_path = f"results/selectqaoa/{version}/{sir_program}-data.json"
        qaoa_times = read_json_field(qaoa_path, "all_qpu_run_times(ms)")
        if not qaoa_times or len(qaoa_times) < 30:
            continue  # salta se dati insufficienti o assenti

        if len(qaoa_times) % 10 != 0:
            continue  # sicurezza: deve essere divisibile per 10

        # Dividi in gruppi da 10 elementi consecutivi e calcola le medie
        grouped_means = [
            np.mean(qaoa_times[i:i+10])
            for i in range(0, len(qaoa_times), 10)
        ]

        if is_not_normal(grouped_means):
            not_normal = True

    status = "not normal" if not_normal else "normal"
    print(f"{sir_program} -> {status}")

## Single-Objective SelectQAOA
Since the quantum state-of-the-art methods IGDec-QAOA, BootQA, and Single-Objective SelectQA formalize the Test Case Selection problem as a single-objective problem with different objectives than those used by the previously used Multi-Objective algorithms, we implemented the Single-Objective SelectQAOA version. 

### Chosen Datasets

We used datasets chosen for the quantum state-of-the-art original works:

- PaintControl: dataset gathered from ABB Robotics Norway;
- GSDTSR: dataset gathered from Google;
- IOF/ROL: industrial datasets for testing complex industrial robots from ABB Robotic;
- ELEVATOR: dataset of tests for elevators from Orona.

 From datasets PaintControl, GSDTSR, and IOF/ROL we read:
 
- "execution time": time needed by the test case to be executed (to minimize);
- "failure rate": the ability, in percentage, of a test case to spot a failure (to maximize).

From ELEVATOR, to form two TCS problems, we read:

- ("ELEVATOR" problem) "execution time" to minimize and "input diversity" (the diversity of the test) to maximize;
- ("ELEVATOR2" problem) execution time" to minimize, "passengers count" (the number of passengers for that case), and "travel distance" (distance crossed by the elevator for the test) both to maximize.

In [ ]:
bootqa_programs = ["gsdtsr","paintcontrol", "iofrol", "elevator", "elevator2"]
bootqa_programs_rep_values = {"gsdtsr":1,"paintcontrol":1,"iofrol":2, "elevator":4, "elevator2":4}
experiments = 10

In [ ]:
def get_data(data_name):
    """Read the datasets"""
    if data_name == "elevator":
        data = pd.read_csv("datasets/quantum_sota_datasets/elevator.csv", dtype={"cost": int, "input_div": float})
    elif data_name == "elevator2":
        data = pd.read_csv("datasets/quantum_sota_datasets/elevator.csv", dtype={"cost": int, "pcount": int, "dist": int})
    else:
        data = pd.read_csv("datasets/quantum_sota_datasets/" + data_name + ".csv", dtype={"time": float, "rate": float})
        data = data[data['rate'] > 0]
    return data

## Problem Decomposition with Clustering

In [ ]:
bootqa_clusters = dict()

for bootqa_program in bootqa_programs:
    data = get_data(bootqa_program)
    
    # Total suite metrics
    if bootqa_program == "elevator" or bootqa_program == "elevator2":
        test_cases_costs = data["cost"].tolist()
    else:
        test_cases_costs = data["time"].tolist()
    
    if bootqa_program == "elevator":
        test_cases_effectiveness = data["input_div"].tolist()
        print(f"Tot suite cost: {sum(test_cases_costs)}")
        print(f"Tot suite input div: {sum(test_cases_effectiveness)}")
    elif bootqa_program == "elevator2":
        test_cases_pcount = data["pcount"].tolist()
        test_cases_dist = data["dist"].tolist()
        print(f"Tot suite cost: {sum(test_cases_costs)}")
        print(f"Tot suite pcount: {sum(test_cases_pcount)}")
        print(f"Tot suite dist: {sum(test_cases_dist)}")
    else:
        test_cases_effectiveness = data["rate"].tolist()
        print(f"Tot suite cost: {sum(test_cases_costs)}")
        print(f"Tot suite rate: {sum(test_cases_effectiveness)}")
        
    # Normalize data
    if bootqa_program != "elevator2":
        cluster_data = np.column_stack((test_cases_costs, test_cases_effectiveness))
    else:
        cluster_data = np.column_stack((test_cases_costs, test_cases_pcount, test_cases_dist))
    
    scaler = StandardScaler()
    normalized_data = scaler.fit_transform(cluster_data)
    
    if bootqa_program == "elevator" or bootqa_program == "elevator2":
        num_clusters = 90
    if bootqa_program == "gsdtsr":
        num_clusters = 50
    if bootqa_program == "iofrol":
        num_clusters = 6
    if bootqa_program == "paintcontrol":
        num_clusters = 4
        
    max_cluster_dim = 5
    
    # Step 2: Perform K-Means Clustering
    start = time.time()
    linkage_matrix = linkage(normalized_data, method='ward')
    clusters = fcluster(linkage_matrix, t=num_clusters, criterion='maxclust')
    
    # Organize test cases by cluster
    clustered_data = defaultdict(list)
    for idx, cluster_id in enumerate(clusters):
        clustered_data[cluster_id].append(idx)
    
    # Process clusters to ensure none exceed max_cluster_dim
    new_cluster_id = max(clustered_data.keys()) + 1  # Start new IDs after existing ones
    to_add = []  # Collect new smaller clusters
    
    for cluster_id, elements in list(clustered_data.items()):  # Avoid modifying dict during iteration
        if len(elements) > max_cluster_dim:
            num_splits = -(-len(elements) // max_cluster_dim)  # Ceiling division to get the required number of splits
            split_size = -(-len(elements) // num_splits)  # Recalculate to distribute elements evenly
            
            # Split while keeping sizes balanced
            parts = [elements[i:i + split_size] for i in range(0, len(elements), split_size)]
    
            # Ensure all new clusters are within max_cluster_dim
            for part in parts:
                if len(part) > max_cluster_dim:
                    raise ValueError(f"A split cluster still exceeds max_cluster_dim ({len(part)} > {max_cluster_dim})!")
    
            # Add new parts to the new clusters
            to_add.extend(parts)
    
            # Remove original large cluster
            del clustered_data[cluster_id]
    
    # Assign new IDs to split parts
    for part in to_add:
        if part:  # Only add if the part is non-empty
            clustered_data[new_cluster_id] = part
            new_cluster_id += 1
    end = time.time()
    print("SelectQAOA Decomposition Time(ms): " + str((end-start)*1000))
    
    bootqa_clusters[bootqa_program] = clustered_data
    
    # Step 3: Calculate the metrics for each refined cluster
    cluster_metrics = {}
    for cluster_id, members in clustered_data.items():
        tot_cluster_costs = sum(test_cases_costs[i] for i in members)
        if bootqa_program != "elevator2":
            tot_cluster_effectiveness = sum(test_cases_effectiveness[i] for i in members)
        else:
            tot_cluster_pcount = sum(test_cases_pcount[i] for i in members)
            tot_cluster_dist = sum(test_cases_dist[i] for i in members)
        if bootqa_program != "elevator2":
            cluster_metrics[cluster_id] = {
                "tot_cluster_cost": tot_cluster_costs,
                "tot_cluster_rates": tot_cluster_effectiveness
            }
        else:
            cluster_metrics[cluster_id] = {
                "tot_cluster_cost": tot_cluster_costs,
                "tot_cluster_pcount": tot_cluster_pcount,
                "tot_cluster_dist": tot_cluster_dist
            }
        print(f"Cluster {cluster_id + 1} metrics:")
        print(f"Test Cases: {members}")
        print(f" - Num. Test Cases: {len(members):.2f}")
        print(f" - Execution Cost: {tot_cluster_costs:.2f}")
        if bootqa_program != "elevator2":
            print(f" - Failure Rate: {tot_cluster_effectiveness}")
        else:
            print(f" - PCount: {tot_cluster_pcount}")
            print(f" - Dist: {tot_cluster_dist}")
    
    print("===========================================================================")    
    
    for cluster_id in clustered_data.keys():
        if len(clustered_data[cluster_id]) > max_cluster_dim:
            print("Program: " + bootqa_program)
            print("Test cases of cluster " + str(cluster_id) + ": " + str(len(clustered_data[cluster_id])))
    
    # Plotting the clusters in 3D space
    fig = plt.figure(figsize=(10, 7))
    if bootqa_program != "elevator2":
        ax = fig.add_subplot(111)
    else:
        ax = fig.add_subplot(111, projection='3d')
    
    # Extracting data for plotting
    exec_costs = np.array(test_cases_costs)
    if bootqa_program != "elevator2":
        effectiveness = np.array(test_cases_effectiveness)
    else:
        pcounts = np.array(test_cases_pcount)
        dists = np.array(test_cases_dist)
    
    # Plot each refined cluster with a different color
    colors = plt.cm.get_cmap("tab10", len(clustered_data))  # A colormap with as many colors as clusters
    for cluster_id, members in clustered_data.items():
        if bootqa_program != "elevator2":
            ax.scatter(
                exec_costs[members], 
                effectiveness[members], 
                color=colors(cluster_id % 10), 
                label=f"Cluster {cluster_id + 1}"
            )
        else:
            ax.scatter(
                exec_costs[members], 
                pcounts[members], 
                dists[members],
                color=colors(cluster_id % 10), 
                label=f"Cluster {cluster_id + 1}"
            )
    
    # Label the axes
    ax.set_xlabel("Execution Cost")
    if bootqa_program != "elevator2":
        ax.set_ylabel("Effectiveness")
    else:
        ax.set_ylabel("Passengers Count")
        ax.set_zlabel("Travel Distance")
    ax.legend()
    ax.set_title("Test Case Clustering Visualization for: " + bootqa_program)
    
    # Display the plot
    plt.show()


## QUBO Problems Definition

In [ ]:
def make_linear_terms_bootqa(cluster_test_cases, test_cases_costs, test_cases_rates, alpha):
    """Making the linear terms of the QUBO"""
    max_cost = max(test_cases_costs)
    
    estimated_costs = []

    #linear coefficients, that are the diagonal of the matrix encoding the QUBO
    for test_case in cluster_test_cases:
        estimated_costs.append((alpha * ((test_cases_costs[test_case])/max_cost)) - ((1-alpha)*test_cases_rates[test_case]))
    
    return np.array(estimated_costs)

def make_linear_terms_bootqa2(cluster_test_cases, test_cases_costs, pcount, dist, alpha):
    """Making the linear terms of the QUBO for the elevator2 problem"""
    max_cost = max(test_cases_costs)
    max_pcount = max(pcount)
    max_dist = max(dist)
    
    estimated_costs = []

    #linear coefficients, that are the diagonal of the matrix encoding the QUBO
    for test_case in cluster_test_cases:
        estimated_costs.append(((alpha/3) * ((test_cases_costs[test_case])/max_cost)) - ((alpha/3) * ((pcount[test_case])/max_pcount)) - ((alpha/3) * ((dist[test_case])/max_dist)))
    
    return np.array(estimated_costs)

In [ ]:
def create_linear_qubo(linear_terms):
    """This function is the one that has to encode the QUBO problem that QAOA will have to solve. The QUBO problem specifies the optimization to solve and a quadratic binary unconstrained problem"""
    qubo = QuadraticProgram()
    
    for i in range(0,len(linear_terms)):
        qubo.binary_var('x%s' % (i))

    qubo.minimize(linear=linear_terms)

    return qubo

## Purpose of Bootstrap Sampling in QAOA Runtime Analysis

Bootstrap sampling is used to estimate the variability and confidence intervals of QAOA execution times when only a limited number of runtime measurements are available. By resampling the observed data with replacement, we can generate an empirical distribution of statistics (e.g., mean or variance), enabling more robust comparisons and uncertainty quantification in performance evaluations.


In [ ]:
def bootstrap_confidence_interval(data, num_samples, confidence_alpha=0.95):
    """This function determines the statistical range within we would expect the mean value of execution times to fall; it relies on the bootstrapping strategy, which allows the calculation of the confidence interval by repeated sampling (with replacement) from the existing data to obtain an estimate of the confidence interval."""
    sample_means = []
    for _ in range(num_samples):
        bootstrap_sample = [random.choice(data) for _ in range(len(data))]
        sample_mean = np.mean(bootstrap_sample)
        sample_means.append(sample_mean)
    
    lower_percentile = (1 - confidence_alpha) / 2 * 100
    upper_percentile = (confidence_alpha + (1 - confidence_alpha) / 2) * 100
    lower_bound = np.percentile(sample_means, lower_percentile)
    upper_bound = np.percentile(sample_means, upper_percentile)
    
    return lower_bound, upper_bound


## Optimal Single-Objective Depth Configuration

## Statevector Simulator

In [ ]:
bootqa_alphas = {"gsdtsr": 0.2,"paintcontrol": 0.80, "iofrol": 0.82, "elevator": 0.50, "elevator2": 2.9}
run_times_dictionary = {"gsdtsr": [],"paintcontrol": [], "iofrol": [], "elevator": [], "elevator2": []}

algorithm_globals.random_seed = 10598
backend = Aer.get_backend("statevector_simulator")

for bootqa_program in bootqa_programs:
    for rep in [1,2,4,8,16]:
        qaoa_mes = QAOA(sampler=BackendSampler(backend=backend), optimizer=COBYLA(100), reps=rep)
        qaoa = MinimumEigenOptimizer(qaoa_mes)
        data = get_data(bootqa_program)
        # Total suite metrics
        if bootqa_program != "elevator2" and bootqa_program != "elevator":
            test_cases_costs = data["time"].tolist()
        else:
            test_cases_costs = data["cost"].tolist()
        test_cases_effectiveness = None
        test_cases_pcount = None
        test_cases_dist = None
        if bootqa_program != "elevator2":
            test_cases_effectiveness = data["input_div"].tolist() if bootqa_program == "elevator" else data["rate"].tolist()
        else:
            test_cases_pcount = data["pcount"].tolist()
            test_cases_dist = data["dist"].tolist()
        
        final_test_suite_costs = []
        final_effectivenesses = []
        final_pcounts = []
        final_dists = []
        for i in range(experiments):
            final_selected_cases = []
            cluster_number = 0
            for cluster_id in bootqa_clusters[bootqa_program]:
                print("Cluster: " + str(bootqa_clusters[bootqa_program][cluster_id]))
                linear_terms = None
                if bootqa_program != "elevator2":
                    linear_terms = make_linear_terms_bootqa(bootqa_clusters[bootqa_program][cluster_id], test_cases_costs, test_cases_effectiveness, bootqa_alphas[bootqa_program])
                else:
                    linear_terms = make_linear_terms_bootqa2(bootqa_clusters[bootqa_program][cluster_id], test_cases_costs,test_cases_pcount, test_cases_dist, bootqa_alphas[bootqa_program])
                linear_qubo = create_linear_qubo(linear_terms)
                print("Linear QUBO: " + str(linear_qubo))
                #for each iteration get the result
                s = time.time()
                qaoa_result = qaoa.solve(linear_qubo)
                e = time.time()
                print("QAOA Result: " + str(qaoa_result))
                run_times_dictionary[bootqa_program].append((e-s)*1000)
                
                variable_values = qaoa_result.x
                indexes_selected_cases = [index for index, value in enumerate(variable_values) if value == 1]
                print("Indexes of selected tests to convert. " + str(indexes_selected_cases))
                selected_tests = []
                for index in indexes_selected_cases:
                    selected_tests.append(bootqa_clusters[bootqa_program][cluster_id][index])
                print("Selected tests: " + str(selected_tests))
                for selected_test in selected_tests:
                    if selected_test not in final_test_suite_costs:
                        final_selected_cases.append(selected_test)
                
            #compute the final test suite cost
            final_test_suite_cost = 0
            for selected_test_case in final_selected_cases:
                final_test_suite_cost += test_cases_costs[selected_test_case]
            final_test_suite_costs.append(final_test_suite_cost)
                
            #compute the total effectiveness
            if bootqa_program != "elevator2":
                final_effectiveness = 0
                for selected_test_case in final_selected_cases:
                    final_effectiveness += test_cases_effectiveness[selected_test_case]
                final_effectivenesses.append(final_effectiveness)
            else:
                final_pcount = 0
                for selected_test_case in final_selected_cases:
                    final_pcount += test_cases_pcount[selected_test_case]
                final_pcounts.append(final_pcount)
                
                final_dist = 0
                for selected_test_case in final_selected_cases:
                    final_dist += test_cases_dist[selected_test_case]
                final_dists.append(final_dist)
        
        print("Final Test Suite: " + str(final_selected_cases))
        #compute the qpu access times
        qpu_run_times_without_zeros = []
        for access_time in run_times_dictionary[bootqa_program]:
          if access_time != 0:
            qpu_run_times_without_zeros.append(access_time)
        lower_bound, upper_bound = bootstrap_confidence_interval(qpu_run_times_without_zeros, 1000, 0.95)
        for i in range(len(run_times_dictionary[bootqa_program])):
          if run_times_dictionary[bootqa_program][i] == 0:
              run_times_dictionary[bootqa_program][i] = upper_bound
        average_qpu_access_time = statistics.mean(run_times_dictionary[bootqa_program]) 
        
        if bootqa_program == "elevator2":
            var_names = ["final_test_suite_costs", "final_pcounts", "final_dists",
                     "average_qpu_access_time(ms)", "stdev_qpu_access_time(ms)", "all_qpu_access_times(ms)",
                     "qpu_lower_bound(ms)", "qpu_upper_bound(ms)", "qpu_run_times(ms)"]
            values = [final_test_suite_costs, final_pcounts, final_dists, average_qpu_access_time, 
                      statistics.stdev(run_times_dictionary[bootqa_program]), run_times_dictionary[bootqa_program],
                      lower_bound, upper_bound, run_times_dictionary[bootqa_program]]
        else:
            var_names = ["final_test_suite_costs", "final_effectivenesses",
                         "average_qpu_access_time(ms)", "stdev_qpu_access_time(ms)", "all_qpu_access_times(ms)",
                         "qpu_lower_bound(ms)", "qpu_upper_bound(ms)", "qpu_run_times(ms)"]
            values = [final_test_suite_costs, final_effectivenesses, average_qpu_access_time, 
                      statistics.stdev(run_times_dictionary[bootqa_program]), run_times_dictionary[bootqa_program],
                      lower_bound, upper_bound, run_times_dictionary[bootqa_program]]
        
        # Ensure the directory exists
        output_dir = "results/selectqaoa/statevector_sim"
        os.makedirs(output_dir, exist_ok=True)
        
        # Path to save the file
        file_path = os.path.join(output_dir, f"{bootqa_program}-rep-{rep}.csv")
        
        # Writing results to the file
        with open(file_path, "w", newline="") as file:
            writer = csv.writer(file)
            writer.writerow(var_names)
            writer.writerow(values)
        print(f"Results saved to {file_path}")

In [ ]:
bootqa_alphas = {"gsdtsr": 0.2,"paintcontrol": 0.80, "iofrol": 0.82, "elevator": 0.50, "elevator2": 2.9}
min_f_vals = {"gsdtsr":None,"paintcontrol":None,"iofrol":None,"elevator":None, "elevator2":None}
normalized_fvals = {}
normality_results = {}

for bootqa_program in bootqa_programs:
    print("Dataset:", bootqa_program)
    
    # Load files
    file_paths = {
        1: f"./results/selectqaoa/statevector_sim/{bootqa_program}-rep-1.csv",
        2: f"./results/selectqaoa/statevector_sim/{bootqa_program}-rep-2.csv",
        4: f"./results/selectqaoa/statevector_sim/{bootqa_program}-rep-4.csv",
        8: f"./results/selectqaoa/statevector_sim/{bootqa_program}-rep-8.csv",
        16: f"./results/selectqaoa/statevector_sim/{bootqa_program}-rep-16.csv"
    }

    dfs = {k: pd.read_csv(v) for k, v in file_paths.items()}
    
    # Parse results
    def extract_column(col):
        return {k: ast.literal_eval(df[col].iloc[-1]) for k, df in dfs.items()}
    
    cost_dict = extract_column('final_test_suite_costs')
    
    if bootqa_program != "elevator2":
        metric_dicts = {
            "final_test_suite_costs": cost_dict,
            "final_effectivenesses": extract_column('final_effectivenesses')
        }
    else:
        metric_dicts = {
            "final_test_suite_costs": cost_dict,
            "final_pcounts": extract_column('final_pcounts'),
            "final_dists": extract_column('final_dists')
        }
    
    alpha = bootqa_alphas[bootqa_program]
    program_fvals = {}

    all_f_vals = []

    # First: compute all f_vals and store them to later get the global min
    temp_fvals = {}
    for version in [1, 2, 4, 8, 16]:
        #print("Version: " + str(version))
        version_fvals = []
        for i in range(10):
            #print("Experiment: " + str(i))
            if bootqa_program != "elevator2":
                cost = metric_dicts["final_test_suite_costs"][version][i]
                fail_rate = metric_dicts["final_effectivenesses"][version][i]
                #print("cost: " + str(cost))
                #print("fail rate: " + str(fail_rate))
                f_val = alpha * cost - (1 - alpha) * fail_rate
                #print("f_val: " + str(f_val))
            else:
                cost = metric_dicts["final_test_suite_costs"][version][i]
                pcount = metric_dicts["final_pcounts"][version][i]
                dist = metric_dicts["final_dists"][version][i]
                #print("cost: " + str(cost))
                #print("pcounts: " + str(pcount))
                #print("final_dists: " + str(dist))
                f_val = (alpha / 3) * (cost - pcount - dist)
                #print("f_val: " + str(f_val))

            version_fvals.append(f_val)
            all_f_vals.append(f_val)
        temp_fvals[version] = version_fvals

    min_f_val = min(all_f_vals)
    print("MIN:" + str(min_f_val))
    
    program_normality = {}
    # Now normalize
    for version in [1, 2, 4, 8, 16]:
        program_fvals[str(version)] = {}
        norm_vals = []
        for i, f_val in enumerate(temp_fvals[version]):
            normalized = f_val / min_f_val
            program_fvals[str(version)][str(i)] = normalized
            norm_vals.append(normalized)
            
        # Test di normalità
        #print(norm_vals)
        stat, p_value = shapiro(norm_vals)
        is_normal = p_value > 0.05
        program_normality[str(version)] = {
            "p_value": round(p_value, 4),
            "is_normal": is_normal
        }
        print(f"{bootqa_program} | rep={version} | p={p_value:.4f} | Normal? {'Yes' if is_normal else 'No'}")

    normalized_fvals[bootqa_program] = program_fvals

# Save to JSON
with open("normalized_fvals.json", "w") as f:
    json.dump(normalized_fvals, f, indent=2)


In [ ]:
"""
GSDTSR BEST rep = 1
PaintControl BEST rep = 1
IOF/ROL BEST rep = 2!
ELEVATOR BEST rep = 4!
ELEVATOR2 BEST rep = 4!
"""

### Aer Simulator

In [ ]:
bootqa_alphas = {"gsdtsr": 0.2,"paintcontrol": 0.80, "iofrol": 0.82, "elevator": 0.50, "elevator2": 2.9}
run_times_dictionary = {"gsdtsr": [],"paintcontrol": [], "iofrol": [], "elevator": [], "elevator2": []}

sim_ideal = AerSimulator()
algorithm_globals.random_seed = 10598

for bootqa_program in bootqa_programs:
    qaoa_mes = QAOA(sampler=BackendSampler(backend=sim_ideal), optimizer=COBYLA(100), reps=bootqa_programs_rep_values[bootqa_program])
    qaoa = MinimumEigenOptimizer(qaoa_mes)  # using QAOA
    data = get_data(bootqa_program)
    # Total suite metrics
    if bootqa_program != "elevator2" and bootqa_program != "elevator":
        test_cases_costs = data["time"].tolist()
    else:
        test_cases_costs = data["cost"].tolist()
    test_cases_rates = None
    test_cases_pcount = None
    test_cases_dist = None
    if bootqa_program != "elevator2":
        test_cases_rates = data["input_div"].tolist() if bootqa_program == "elevator" else data["rate"].tolist()
    else:
        test_cases_pcount = data["pcount"].tolist()
        test_cases_dist = data["dist"].tolist()
    
    final_test_suite_costs = []
    final_effectivenesses = []
    final_pcounts = []
    final_dists = []
    for i in range(experiments):
        final_selected_cases = []
        cluster_number = 0
        for cluster_id in bootqa_clusters[bootqa_program]:
            print("Cluster: " + str(bootqa_clusters[bootqa_program][cluster_id]))
            linear_terms = None
            if bootqa_program != "elevator2":
                linear_terms = make_linear_terms_bootqa(bootqa_clusters[bootqa_program][cluster_id], test_cases_costs, test_cases_rates, bootqa_alphas[bootqa_program])
            else:
                linear_terms = make_linear_terms_bootqa2(bootqa_clusters[bootqa_program][cluster_id], test_cases_costs,test_cases_pcount, test_cases_dist, bootqa_alphas[bootqa_program])
            linear_qubo = create_linear_qubo(linear_terms)
            print("Linear QUBO: " + str(linear_qubo))
            #for each iteration get the result
            s = time.time()
            qaoa_result = qaoa.solve(linear_qubo)
            e = time.time()
            print("QAOA Result: " + str(qaoa_result))
            run_times_dictionary[bootqa_program].append((e-s)*1000)
            
            variable_values = qaoa_result.x
            indexes_selected_cases = [index for index, value in enumerate(variable_values) if value == 1]
            print("Indexes of selected tests to convert. " + str(indexes_selected_cases))
            selected_tests = []
            for index in indexes_selected_cases:
                selected_tests.append(bootqa_clusters[bootqa_program][cluster_id][index])
            print("Selected tests: " + str(selected_tests))
            for selected_test in selected_tests:
                if selected_test not in final_test_suite_costs:
                    final_selected_cases.append(selected_test)
            
        #compute the final test suite cost
        final_test_suite_cost = 0
        for selected_test_case in final_selected_cases:
            final_test_suite_cost += test_cases_costs[selected_test_case]
        final_test_suite_costs.append(final_test_suite_cost)
            
        #compute the total failure rate
        if bootqa_program != "elevator2":
            final_effectiveness = 0
            for selected_test_case in final_selected_cases:
                final_effectiveness += test_cases_rates[selected_test_case]
            final_effectivenesses.append(final_effectiveness)
        else:
            final_pcount = 0
            for selected_test_case in final_selected_cases:
                final_pcount += test_cases_pcount[selected_test_case]
            final_pcounts.append(final_pcount)
            
            final_dist = 0
            for selected_test_case in final_selected_cases:
                final_dist += test_cases_dist[selected_test_case]
            final_dists.append(final_dist)
    
    print("Final Test Suite: " + str(final_selected_cases))
    #compute the qpu access times
    qpu_run_times_without_zeros = []
    for access_time in run_times_dictionary[bootqa_program]:
      if access_time != 0:
        qpu_run_times_without_zeros.append(access_time)
    lower_bound, upper_bound = bootstrap_confidence_interval(qpu_run_times_without_zeros, 1000, 0.95)
    for i in range(len(run_times_dictionary[bootqa_program])):
      if run_times_dictionary[bootqa_program][i] == 0:
          run_times_dictionary[bootqa_program][i] = upper_bound
    average_qpu_access_time = statistics.mean(run_times_dictionary[bootqa_program]) 
    
    if bootqa_program == "elevator2":
        var_names = ["final_test_suite_costs", "final_pcounts", "final_dists",
                 "average_qpu_access_time(ms)", "stdev_qpu_access_time(ms)", "all_qpu_access_times(ms)",
                 "qpu_lower_bound(ms)", "qpu_upper_bound(ms)", "qpu_run_times(ms)"]
        values = [final_test_suite_costs, final_pcounts, final_dists, average_qpu_access_time, 
                  statistics.stdev(run_times_dictionary[bootqa_program]), run_times_dictionary[bootqa_program],
                  lower_bound, upper_bound, run_times_dictionary[bootqa_program]]
    else:
        var_names = ["final_test_suite_costs", "final_effectivenesses",
                     "average_qpu_access_time(ms)", "stdev_qpu_access_time(ms)", "all_qpu_access_times(ms)",
                     "qpu_lower_bound(ms)", "qpu_upper_bound(ms)", "qpu_run_times(ms)"]
        values = [final_test_suite_costs, final_effectivenesses, average_qpu_access_time, 
                  statistics.stdev(run_times_dictionary[bootqa_program]), run_times_dictionary[bootqa_program],
                  lower_bound, upper_bound, run_times_dictionary[bootqa_program]]
    
    # Ensure the directory exists
    output_dir = "results/selectqaoa/aer_sim"
    os.makedirs(output_dir, exist_ok=True)
    
    # Path to save the file
    file_path = os.path.join(output_dir, f"{bootqa_program}.csv")
    
    # Writing results to the file
    with open(file_path, "w", newline="") as file:
        writer = csv.writer(file)
        writer.writerow(var_names)
        writer.writerow(values)
    print(f"Results saved to {file_path}")

## Fake Vigo Noise Simulator

In [ ]:
bootqa_alphas = {"gsdtsr": 0.2,"paintcontrol": 0.80, "iofrol": 0.82, "elevator": 0.50, "elevator2": 2.9}
run_times_dictionary = {"gsdtsr": [],"paintcontrol": [], "iofrol": [], "elevator": [], "elevator2": []}

device_backend = FakeVigoV2()

algorithm_globals.random_seed = 10598

for bootqa_program in bootqa_programs:
    qaoa_mes = QAOA(sampler=BackendSampler(backend=device_backend), optimizer=COBYLA(100), reps=bootqa_programs_rep_values[bootqa_program])
    qaoa = MinimumEigenOptimizer(qaoa_mes)  # using QAOA
    data = get_data(bootqa_program)
    # Total suite metrics
    if bootqa_program != "elevator2" and bootqa_program != "elevator":
        test_cases_costs = data["time"].tolist()
    else:
        test_cases_costs = data["cost"].tolist()
    test_cases_rates = None
    test_cases_pcount = None
    test_cases_dist = None
    if bootqa_program != "elevator2":
        test_cases_rates = data["input_div"].tolist() if bootqa_program == "elevator" else data["rate"].tolist()
    else:
        test_cases_pcount = data["pcount"].tolist()
        test_cases_dist = data["dist"].tolist()
    
    final_test_suite_costs = []
    final_effectivenesses = []
    final_pcounts = []
    final_dists = []
    for i in range(experiments):
        final_selected_cases = []
        cluster_number = 0
        for cluster_id in bootqa_clusters[bootqa_program]:
            print("Cluster: " + str(bootqa_clusters[bootqa_program][cluster_id]))
            linear_terms = None
            if bootqa_program != "elevator2":
                linear_terms = make_linear_terms_bootqa(bootqa_clusters[bootqa_program][cluster_id], test_cases_costs, test_cases_rates, bootqa_alphas[bootqa_program])
            else:
                linear_terms = make_linear_terms_bootqa2(bootqa_clusters[bootqa_program][cluster_id], test_cases_costs,test_cases_pcount, test_cases_dist, bootqa_alphas[bootqa_program])
            linear_qubo = create_linear_qubo(linear_terms)
            print("Linear QUBO: " + str(linear_qubo))
            #for each iteration get the result
            s = time.time()
            qaoa_result = qaoa.solve(linear_qubo)
            e = time.time()
            print("Program: " + str(bootqa_program) + ". It: " + str(i))
            print("QAOA Result: " + str(qaoa_result))
            run_times_dictionary[bootqa_program].append((e-s)*1000)
            
            variable_values = qaoa_result.x
            indexes_selected_cases = [index for index, value in enumerate(variable_values) if value == 1]
            print("Indexes of selected tests to convert. " + str(indexes_selected_cases))
            selected_tests = []
            for index in indexes_selected_cases:
                selected_tests.append(bootqa_clusters[bootqa_program][cluster_id][index])
            print("Selected tests: " + str(selected_tests))
            for selected_test in selected_tests:
                if selected_test not in final_test_suite_costs:
                    final_selected_cases.append(selected_test)
            
        #compute the final test suite cost
        final_test_suite_cost = 0
        for selected_test_case in final_selected_cases:
            final_test_suite_cost += test_cases_costs[selected_test_case]
        final_test_suite_costs.append(final_test_suite_cost)
            
        #compute the total failure rate
        if bootqa_program != "elevator2":
            final_effectiveness = 0
            for selected_test_case in final_selected_cases:
                final_effectiveness += test_cases_rates[selected_test_case]
            final_effectivenesses.append(final_effectiveness)
        else:
            final_pcount = 0
            for selected_test_case in final_selected_cases:
                final_pcount += test_cases_pcount[selected_test_case]
            final_pcounts.append(final_pcount)
            
            final_dist = 0
            for selected_test_case in final_selected_cases:
                final_dist += test_cases_dist[selected_test_case]
            final_dists.append(final_dist)
    
    print("Final Test Suite: " + str(final_selected_cases))
    #compute the qpu access times
    qpu_run_times_without_zeros = []
    for access_time in run_times_dictionary[bootqa_program]:
      if access_time != 0:
        qpu_run_times_without_zeros.append(access_time)
    lower_bound, upper_bound = bootstrap_confidence_interval(qpu_run_times_without_zeros, 1000, 0.95)
    for i in range(len(run_times_dictionary[bootqa_program])):
      if run_times_dictionary[bootqa_program][i] == 0:
          run_times_dictionary[bootqa_program][i] = upper_bound
    average_qpu_access_time = statistics.mean(run_times_dictionary[bootqa_program]) 
    
    if bootqa_program == "elevator2":
        var_names = ["final_test_suite_costs", "final_pcounts", "final_dists",
                 "average_qpu_access_time(ms)", "stdev_qpu_access_time(ms)", "all_qpu_access_times(ms)",
                 "qpu_lower_bound(ms)", "qpu_upper_bound(ms)", "qpu_run_times(ms)"]
        values = [final_test_suite_costs, final_pcounts, final_dists, average_qpu_access_time, 
                  statistics.stdev(run_times_dictionary[bootqa_program]), run_times_dictionary[bootqa_program],
                  lower_bound, upper_bound, run_times_dictionary[bootqa_program]]
    else:
        var_names = ["final_test_suite_costs", "final_effectivenesses",
                     "average_qpu_access_time(ms)", "stdev_qpu_access_time(ms)", "all_qpu_access_times(ms)",
                     "qpu_lower_bound(ms)", "qpu_upper_bound(ms)", "qpu_run_times(ms)"]
        values = [final_test_suite_costs, final_effectivenesses, average_qpu_access_time, 
                  statistics.stdev(run_times_dictionary[bootqa_program]), run_times_dictionary[bootqa_program],
                  lower_bound, upper_bound, run_times_dictionary[bootqa_program]]
    
    # Ensure the directory exists
    output_dir = "results/selectqaoa/fake_vigo"
    os.makedirs(output_dir, exist_ok=True)
    
    # Path to save the file
    file_path = os.path.join(output_dir, f"{bootqa_program}.csv")
    
    # Writing results to the file
    with open(file_path, "w", newline="") as file:
        writer = csv.writer(file)
        writer.writerow(var_names)
        writer.writerow(values)
    print(f"Results saved to {file_path}")

## Noise Mitigation in Quantum Computing

### Zero-Noise Extrapolation (ZNE)

In [ ]:
def expectation_from_distribution(distribution, operator):
    """Calcola ⟨H⟩ da distribuzione e SparsePauliOp (solo Z/I)."""
    expectation = 0.0
    for pauli, coeff in zip(operator.paulis, operator.coeffs):
        pauli_str = pauli.to_label()
        if any(p in pauli_str for p in "XY"):  # ignora se contiene X o Y
            continue
        for bitstring, prob in distribution.items():
            if len(pauli_str) < len(bitstring):
                pauli_str = pauli_str.rjust(len(bitstring), "I")
            parity = 1
            for i in range(len(bitstring)):
                p = pauli_str[-(i + 1)]  # allineato da destra (qubit 0 a dx)
                if p == 'Z':
                    parity *= 1 if bitstring[-(i + 1)] == '0' else -1
            expectation += coeff.real * prob * parity
    return expectation.real

# Config
bootqa_alphas = {"gsdtsr": 0.2, "paintcontrol": 0.80, "iofrol": 0.82, "elevator": 0.50, "elevator2": 2.9}
run_times_dictionary = {k: [] for k in bootqa_alphas}
algorithm_globals.random_seed = 10598

scale_factors = [1.0, 3.0, 5.0]
factory = RichardsonFactory(scale_factors=scale_factors)

for bootqa_program in bootqa_programs:
    data = get_data(bootqa_program)
    test_cases_costs = data["time"].tolist() if bootqa_program not in ["elevator", "elevator2"] else data["cost"].tolist()
    if bootqa_program == "elevator":
        test_cases_rates = data["input_div"].tolist()
    elif bootqa_program != "elevator2":
        test_cases_rates = data["rate"].tolist()
    else:
        test_cases_pcount = data["pcount"].tolist()
        test_cases_dist = data["dist"].tolist()

    final_test_suite_costs = []
    final_effectivenesses = []
    final_pcounts = []
    final_dists = []

    for i in range(experiments):
        final_selected_cases = []

        for cluster_id in bootqa_clusters[bootqa_program]:
            cluster = bootqa_clusters[bootqa_program][cluster_id]
            print("Cluster: " + str(bootqa_clusters[bootqa_program][cluster_id]))

            # QUBO creation
            if bootqa_program != "elevator2":
                linear_terms = make_linear_terms_bootqa(cluster, test_cases_costs, test_cases_rates, bootqa_alphas[bootqa_program])
            else:
                linear_terms = make_linear_terms_bootqa2(cluster, test_cases_costs, test_cases_pcount, test_cases_dist, bootqa_alphas[bootqa_program])
            linear_qubo = create_linear_qubo(linear_terms)
            print("Linear QUBO: " + str(linear_qubo))

            # QAOA to build the circuit needs an Ising Hamiltonian as input
            # Since the problem is a QuadraticProgram object, we must convert it into an Ising
            qp2qubo = QuadraticProgramToQubo()
            qubo_problem = qp2qubo.convert(linear_qubo)
            hamiltonian, _ = qubo_problem.to_ising()
            backend = FakeVigoV2()

            # QAOAAnsatz builds a circuit with defined repetitions 
            reps = bootqa_programs_rep_values[bootqa_program]
            ansatz = QAOAAnsatz(hamiltonian, reps=reps)

            # Returns the 0 noise expected value of H with given params
            def zne_expectation(params):
                # Assigns params to γ and β (2*reps params)
                circuit = ansatz.assign_parameters(params)
                circuit.measure_all() # the circuit will produce a result in terms of bitstrings
            
                # Running the circuit on FakeVigo, getting the expected value for Hamiltonian 
                def run_expectation(circ):
                    sampler = BackendSampler(backend=device_backend)
                    result = sampler.run(circ).result()
                    dist = result.quasi_dists[0].binary_probabilities()
                    return expectation_from_distribution(dist, hamiltonian)
            
                # Running the the circuit at different noise levels, to extrapolate 0 noise H exp value
                factory = RichardsonFactory(scale_factors=[1.0, 3.0, 5.0])
                return zne.execute_with_zne(circuit, run_expectation, scale_noise=fold_global, factory=factory)
            
            # Initialize params
            initial_params = np.array([0.5] * ansatz.num_parameters)
            # Tries different params, calling for any trial zne_expectation, updates params based on H expected value retuned any time, until it converges
            result = minimize(zne_expectation, initial_params, method="COBYLA")
            
            # Get the optimal params
            optimal_params = result.x
                
            # Build the final circuit with the optimal params
            final_circuit = ansatz.assign_parameters(optimal_params)
            final_circuit.measure_all()
            
            # Execute the final circuite to get the final result
            sampler = BackendSampler(backend=device_backend)
            start = time.time()
            final_result = sampler.run(final_circuit).result()
            end = time.time()
            counts = final_result.quasi_dists[0].binary_probabilities()
            best_bitstring = max(counts.items(), key=lambda x: x[1])[0]
            
            # Ora puoi estrarre gli indici dei test selezionati
            selected_tests = [cluster[i] for i, b in enumerate(best_bitstring[::-1]) if b == "1"]
            print(f"[{bootqa_program}] Iter {i} - Best bitstring: {best_bitstring} -> {selected_tests}")

            for test in selected_tests:
                if test not in final_selected_cases:
                    final_selected_cases.append(test)

            run_times_dictionary[bootqa_program].append((end-start)*1000)

        cost = sum(test_cases_costs[t] for t in final_selected_cases)
        final_test_suite_costs.append(cost)

        if bootqa_program != "elevator2":
            effectiveness = sum(test_cases_rates[t] for t in final_selected_cases)
            final_effectivenesses.append(effectiveness)
        else:
            pcount = sum(test_cases_pcount[t] for t in final_selected_cases)
            dist = sum(test_cases_dist[t] for t in final_selected_cases)
            final_pcounts.append(pcount)
            final_dists.append(dist)

    avg_time = statistics.mean(run_times_dictionary[bootqa_program]) 
    std_time = statistics.stdev(run_times_dictionary[bootqa_program]) 

    # Export results
    if bootqa_program == "elevator2":
        var_names = ["final_test_suite_costs", "final_pcounts", "final_dists",
                     "average_qpu_access_time(ms)", "stdev_qpu_access_time(ms)",
                     "qpu_run_times(ms)"]
        values = [final_test_suite_costs, final_pcounts, final_dists, avg_time,
                  std_time, run_times_dictionary[bootqa_program]]
    else:
        var_names = ["final_test_suite_costs", "final_effectivenesses",
                     "average_qpu_access_time(ms)", "stdev_qpu_access_time(ms)",
                     "qpu_run_times(ms)"]
        values = [final_test_suite_costs, final_effectivenesses, avg_time,
                  std_time, run_times_dictionary[bootqa_program]]

    os.makedirs("results/selectqaoa/fake_vigo_zne", exist_ok=True)
    file_path = f"results/selectqaoa/fake_vigo_zne/{bootqa_program}.csv"
    with open(file_path, "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(var_names)
        writer.writerow(values)

    print(f"Results saved to {file_path}")


## Depolarizing Error Simulator

In [ ]:
bootqa_alphas = {"gsdtsr": 0.2,"paintcontrol": 0.80, "iofrol": 0.82, "elevator": 0.50, "elevator2": 2.9}
run_times_dictionary = {"gsdtsr": [],"paintcontrol": [], "iofrol": [], "elevator": [], "elevator2": []}

noise_dep = NoiseModel()

# Define a 1% depolarizing error for single-qubit and two-qubit gates
depolarizing_error_1q = depolarizing_error(0.01, 1)  # 1% depolarizing error for single-qubit gates
depolarizing_error_2q = depolarizing_error(0.01, 2)  # 1% depolarizing error for two-qubit gates

# Apply depolarizing noise to single-qubit gates
noise_dep.add_all_qubit_quantum_error(depolarizing_error_1q, ['rx', 'rz', 'sx', 'x', 'h'])

# Apply depolarizing noise to two-qubit gates
noise_dep.add_all_qubit_quantum_error(depolarizing_error_2q, ['cx', 'cz', 'swap'])

sim_dep = AerSimulator(noise_model=noise_dep)

algorithm_globals.random_seed = 10598

for bootqa_program in bootqa_programs:
    qaoa_mes = QAOA(sampler=BackendSampler(backend=sim_dep), optimizer=COBYLA(100), reps=bootqa_programs_rep_values[bootqa_program])
    qaoa = MinimumEigenOptimizer(qaoa_mes)  # using QAOA
    data = get_data(bootqa_program)
    # Total suite metrics
    if bootqa_program != "elevator2" and bootqa_program != "elevator":
        test_cases_costs = data["time"].tolist()
    else:
        test_cases_costs = data["cost"].tolist()
    test_cases_rates = None
    test_cases_pcount = None
    test_cases_dist = None
    if bootqa_program != "elevator2":
        test_cases_rates = data["input_div"].tolist() if bootqa_program == "elevator" else data["rate"].tolist()
    else:
        test_cases_pcount = data["pcount"].tolist()
        test_cases_dist = data["dist"].tolist()
    
    final_test_suite_costs = []
    final_effectivenesses = []
    final_pcounts = []
    final_dists = []
    for i in range(experiments):
        final_selected_cases = []
        cluster_number = 0
        for cluster_id in bootqa_clusters[bootqa_program]:
            print("Cluster: " + str(bootqa_clusters[bootqa_program][cluster_id]))
            linear_terms = None
            if bootqa_program != "elevator2":
                linear_terms = make_linear_terms_bootqa(bootqa_clusters[bootqa_program][cluster_id], test_cases_costs, test_cases_rates, bootqa_alphas[bootqa_program])
            else:
                linear_terms = make_linear_terms_bootqa2(bootqa_clusters[bootqa_program][cluster_id], test_cases_costs,test_cases_pcount, test_cases_dist, bootqa_alphas[bootqa_program])
            linear_qubo = create_linear_qubo(linear_terms)
            print("Linear QUBO: " + str(linear_qubo))
            #for each iteration get the result
            s = time.time()
            qaoa_result = qaoa.solve(linear_qubo)
            e = time.time()
            print("Program: " + str(bootqa_program) + ". It: " + str(i))
            print("QAOA Result: " + str(qaoa_result))
            run_times_dictionary[bootqa_program].append((e-s)*1000)
            
            variable_values = qaoa_result.x
            indexes_selected_cases = [index for index, value in enumerate(variable_values) if value == 1]
            print("Indexes of selected tests to convert. " + str(indexes_selected_cases))
            selected_tests = []
            for index in indexes_selected_cases:
                selected_tests.append(bootqa_clusters[bootqa_program][cluster_id][index])
            print("Selected tests: " + str(selected_tests))
            for selected_test in selected_tests:
                if selected_test not in final_test_suite_costs:
                    final_selected_cases.append(selected_test)
            
        #compute the final test suite cost
        final_test_suite_cost = 0
        for selected_test_case in final_selected_cases:
            final_test_suite_cost += test_cases_costs[selected_test_case]
        final_test_suite_costs.append(final_test_suite_cost)
            
        #compute the total failure rate
        if bootqa_program != "elevator2":
            final_effectiveness = 0
            for selected_test_case in final_selected_cases:
                final_effectiveness += test_cases_rates[selected_test_case]
            final_effectivenesses.append(final_effectiveness)
        else:
            final_pcount = 0
            for selected_test_case in final_selected_cases:
                final_pcount += test_cases_pcount[selected_test_case]
            final_pcounts.append(final_pcount)
            
            final_dist = 0
            for selected_test_case in final_selected_cases:
                final_dist += test_cases_dist[selected_test_case]
            final_dists.append(final_dist)
    
    print("Final Test Suite: " + str(final_selected_cases))
    #compute the qpu access times
    qpu_run_times_without_zeros = []
    for access_time in run_times_dictionary[bootqa_program]:
      if access_time != 0:
        qpu_run_times_without_zeros.append(access_time)
    lower_bound, upper_bound = bootstrap_confidence_interval(qpu_run_times_without_zeros, 1000, 0.95)
    for i in range(len(run_times_dictionary[bootqa_program])):
      if run_times_dictionary[bootqa_program][i] == 0:
          run_times_dictionary[bootqa_program][i] = upper_bound
    average_qpu_access_time = statistics.mean(run_times_dictionary[bootqa_program]) 
    
    if bootqa_program == "elevator2":
        var_names = ["final_test_suite_costs", "final_pcounts", "final_dists",
                 "average_qpu_access_time(ms)", "stdev_qpu_access_time(ms)", "all_qpu_access_times(ms)",
                 "qpu_lower_bound(ms)", "qpu_upper_bound(ms)", "qpu_run_times(ms)"]
        values = [final_test_suite_costs, final_pcounts, final_dists, average_qpu_access_time, 
                  statistics.stdev(run_times_dictionary[bootqa_program]), run_times_dictionary[bootqa_program],
                  lower_bound, upper_bound, run_times_dictionary[bootqa_program]]
    else:
        var_names = ["final_test_suite_costs", "final_effectivenesses",
                     "average_qpu_access_time(ms)", "stdev_qpu_access_time(ms)", "all_qpu_access_times(ms)",
                     "qpu_lower_bound(ms)", "qpu_upper_bound(ms)", "qpu_run_times(ms)"]
        values = [final_test_suite_costs, final_effectivenesses, average_qpu_access_time, 
                  statistics.stdev(run_times_dictionary[bootqa_program]), run_times_dictionary[bootqa_program],
                  lower_bound, upper_bound, run_times_dictionary[bootqa_program]]
    
    # Ensure the directory exists
    output_dir = "results/selectqaoa/depolarizing_sim/01"
    os.makedirs(output_dir, exist_ok=True)
    
    # Path to save the file
    file_path = os.path.join(output_dir, f"{bootqa_program}.csv")
    
    # Writing results to the file
    with open(file_path, "w", newline="") as file:
        writer = csv.writer(file)
        writer.writerow(var_names)
        writer.writerow(values)
    print(f"Results saved to {file_path}")

In [ ]:
bootqa_alphas = {"gsdtsr": 0.2,"paintcontrol": 0.80, "iofrol": 0.82, "elevator": 0.50, "elevator2": 2.9}
run_times_dictionary = {"gsdtsr": [],"paintcontrol": [], "iofrol": [], "elevator": [], "elevator2": []}

noise_dep = NoiseModel()

# Define a 2% depolarizing error for single-qubit and two-qubit gates
depolarizing_error_1q = depolarizing_error(0.02, 1)  # 2% depolarizing error for single-qubit gates
depolarizing_error_2q = depolarizing_error(0.02, 2)  # 2% depolarizing error for two-qubit gates

# Apply depolarizing noise to single-qubit gates
noise_dep.add_all_qubit_quantum_error(depolarizing_error_1q, ['rx', 'rz', 'sx', 'x', 'h'])

# Apply depolarizing noise to two-qubit gates
noise_dep.add_all_qubit_quantum_error(depolarizing_error_2q, ['cx', 'cz', 'swap'])

sim_dep = AerSimulator(noise_model=noise_dep)

algorithm_globals.random_seed = 10598

for bootqa_program in bootqa_programs:
    qaoa_mes = QAOA(sampler=BackendSampler(backend=sim_dep), optimizer=COBYLA(100), reps=bootqa_programs_rep_values[bootqa_program])
    qaoa = MinimumEigenOptimizer(qaoa_mes)  # using QAOA
    data = get_data(bootqa_program)
    # Total suite metrics
    if bootqa_program != "elevator2" and bootqa_program != "elevator":
        test_cases_costs = data["time"].tolist()
    else:
        test_cases_costs = data["cost"].tolist()
    test_cases_rates = None
    test_cases_pcount = None
    test_cases_dist = None
    if bootqa_program != "elevator2":
        test_cases_rates = data["input_div"].tolist() if bootqa_program == "elevator" else data["rate"].tolist()
    else:
        test_cases_pcount = data["pcount"].tolist()
        test_cases_dist = data["dist"].tolist()
    
    final_test_suite_costs = []
    final_effectivenesses = []
    final_pcounts = []
    final_dists = []
    for i in range(experiments):
        final_selected_cases = []
        cluster_number = 0
        for cluster_id in bootqa_clusters[bootqa_program]:
            print("Cluster: " + str(bootqa_clusters[bootqa_program][cluster_id]))
            linear_terms = None
            if bootqa_program != "elevator2":
                linear_terms = make_linear_terms_bootqa(bootqa_clusters[bootqa_program][cluster_id], test_cases_costs, test_cases_rates, bootqa_alphas[bootqa_program])
            else:
                linear_terms = make_linear_terms_bootqa2(bootqa_clusters[bootqa_program][cluster_id], test_cases_costs,test_cases_pcount, test_cases_dist, bootqa_alphas[bootqa_program])
            linear_qubo = create_linear_qubo(linear_terms)
            print("Linear QUBO: " + str(linear_qubo))
            #for each iteration get the result
            s = time.time()
            qaoa_result = qaoa.solve(linear_qubo)
            e = time.time()
            print("Program: " + str(bootqa_program) + ". It: " + str(i))
            print("QAOA Result: " + str(qaoa_result))
            run_times_dictionary[bootqa_program].append((e-s)*1000)
            
            variable_values = qaoa_result.x
            indexes_selected_cases = [index for index, value in enumerate(variable_values) if value == 1]
            print("Indexes of selected tests to convert. " + str(indexes_selected_cases))
            selected_tests = []
            for index in indexes_selected_cases:
                selected_tests.append(bootqa_clusters[bootqa_program][cluster_id][index])
            print("Selected tests: " + str(selected_tests))
            for selected_test in selected_tests:
                if selected_test not in final_test_suite_costs:
                    final_selected_cases.append(selected_test)
            
        #compute the final test suite cost
        final_test_suite_cost = 0
        for selected_test_case in final_selected_cases:
            final_test_suite_cost += test_cases_costs[selected_test_case]
        final_test_suite_costs.append(final_test_suite_cost)
            
        #compute the total failure rate
        if bootqa_program != "elevator2":
            final_effectiveness = 0
            for selected_test_case in final_selected_cases:
                final_effectiveness += test_cases_rates[selected_test_case]
            final_effectivenesses.append(final_effectiveness)
        else:
            final_pcount = 0
            for selected_test_case in final_selected_cases:
                final_pcount += test_cases_pcount[selected_test_case]
            final_pcounts.append(final_pcount)
            
            final_dist = 0
            for selected_test_case in final_selected_cases:
                final_dist += test_cases_dist[selected_test_case]
            final_dists.append(final_dist)
    
    print("Final Test Suite: " + str(final_selected_cases))
    #compute the qpu access times
    qpu_run_times_without_zeros = []
    for access_time in run_times_dictionary[bootqa_program]:
      if access_time != 0:
        qpu_run_times_without_zeros.append(access_time)
    lower_bound, upper_bound = bootstrap_confidence_interval(qpu_run_times_without_zeros, 1000, 0.95)
    for i in range(len(run_times_dictionary[bootqa_program])):
      if run_times_dictionary[bootqa_program][i] == 0:
          run_times_dictionary[bootqa_program][i] = upper_bound
    average_qpu_access_time = statistics.mean(run_times_dictionary[bootqa_program]) 
    
    if bootqa_program == "elevator2":
        var_names = ["final_test_suite_costs", "final_pcounts", "final_dists",
                 "average_qpu_access_time(ms)", "stdev_qpu_access_time(ms)", "all_qpu_access_times(ms)",
                 "qpu_lower_bound(ms)", "qpu_upper_bound(ms)", "qpu_run_times(ms)"]
        values = [final_test_suite_costs, final_pcounts, final_dists, average_qpu_access_time, 
                  statistics.stdev(run_times_dictionary[bootqa_program]), run_times_dictionary[bootqa_program],
                  lower_bound, upper_bound, run_times_dictionary[bootqa_program]]
    else:
        var_names = ["final_test_suite_costs", "final_effectivenesses",
                     "average_qpu_access_time(ms)", "stdev_qpu_access_time(ms)", "all_qpu_access_times(ms)",
                     "qpu_lower_bound(ms)", "qpu_upper_bound(ms)", "qpu_run_times(ms)"]
        values = [final_test_suite_costs, final_effectivenesses, average_qpu_access_time, 
                  statistics.stdev(run_times_dictionary[bootqa_program]), run_times_dictionary[bootqa_program],
                  lower_bound, upper_bound, run_times_dictionary[bootqa_program]]
    
    # Ensure the directory exists
    output_dir = "results/selectqaoa/depolarizing_sim/02"
    os.makedirs(output_dir, exist_ok=True)
    
    # Path to save the file
    file_path = os.path.join(output_dir, f"{bootqa_program}.csv")
    
    # Writing results to the file
    with open(file_path, "w", newline="") as file:
        writer = csv.writer(file)
        writer.writerow(var_names)
        writer.writerow(values)
    print(f"Results saved to {file_path}")

In [ ]:
bootqa_alphas = {"gsdtsr": 0.2,"paintcontrol": 0.80, "iofrol": 0.82, "elevator": 0.50, "elevator2": 2.9}
run_times_dictionary = {"gsdtsr": [],"paintcontrol": [], "iofrol": [], "elevator": [], "elevator2": []}

noise_dep = NoiseModel()

# Define a 5% depolarizing error for single-qubit and two-qubit gates
depolarizing_error_1q = depolarizing_error(0.05, 1)  # 5% depolarizing error for single-qubit gates
depolarizing_error_2q = depolarizing_error(0.05, 2)  # 5% depolarizing error for two-qubit gates

# Apply depolarizing noise to single-qubit gates
noise_dep.add_all_qubit_quantum_error(depolarizing_error_1q, ['rx', 'rz', 'sx', 'x', 'h'])

# Apply depolarizing noise to two-qubit gates
noise_dep.add_all_qubit_quantum_error(depolarizing_error_2q, ['cx', 'cz', 'swap'])

sim_dep = AerSimulator(noise_model=noise_dep)

algorithm_globals.random_seed = 10598

for bootqa_program in bootqa_programs:
    qaoa_mes = QAOA(sampler=BackendSampler(backend=sim_dep), optimizer=COBYLA(100), reps=bootqa_programs_rep_values[bootqa_program])
    qaoa = MinimumEigenOptimizer(qaoa_mes)  # using QAOA
    data = get_data(bootqa_program)
    # Total suite metrics
    if bootqa_program != "elevator2" and bootqa_program != "elevator":
        test_cases_costs = data["time"].tolist()
    else:
        test_cases_costs = data["cost"].tolist()
    test_cases_rates = None
    test_cases_pcount = None
    test_cases_dist = None
    if bootqa_program != "elevator2":
        test_cases_rates = data["input_div"].tolist() if bootqa_program == "elevator" else data["rate"].tolist()
    else:
        test_cases_pcount = data["pcount"].tolist()
        test_cases_dist = data["dist"].tolist()
    
    final_test_suite_costs = []
    final_effectivenesses = []
    final_pcounts = []
    final_dists = []
    for i in range(experiments):
        final_selected_cases = []
        cluster_number = 0
        for cluster_id in bootqa_clusters[bootqa_program]:
            print("Cluster: " + str(bootqa_clusters[bootqa_program][cluster_id]))
            linear_terms = None
            if bootqa_program != "elevator2":
                linear_terms = make_linear_terms_bootqa(bootqa_clusters[bootqa_program][cluster_id], test_cases_costs, test_cases_rates, bootqa_alphas[bootqa_program])
            else:
                linear_terms = make_linear_terms_bootqa2(bootqa_clusters[bootqa_program][cluster_id], test_cases_costs,test_cases_pcount, test_cases_dist, bootqa_alphas[bootqa_program])
            linear_qubo = create_linear_qubo(linear_terms)
            print("Linear QUBO: " + str(linear_qubo))
            #for each iteration get the result
            s = time.time()
            qaoa_result = qaoa.solve(linear_qubo)
            e = time.time()
            print("Program: " + str(bootqa_program) + ". It: " + str(i))
            print("QAOA Result: " + str(qaoa_result))
            run_times_dictionary[bootqa_program].append((e-s)*1000)
            
            variable_values = qaoa_result.x
            indexes_selected_cases = [index for index, value in enumerate(variable_values) if value == 1]
            print("Indexes of selected tests to convert. " + str(indexes_selected_cases))
            selected_tests = []
            for index in indexes_selected_cases:
                selected_tests.append(bootqa_clusters[bootqa_program][cluster_id][index])
            print("Selected tests: " + str(selected_tests))
            for selected_test in selected_tests:
                if selected_test not in final_test_suite_costs:
                    final_selected_cases.append(selected_test)
            
        #compute the final test suite cost
        final_test_suite_cost = 0
        for selected_test_case in final_selected_cases:
            final_test_suite_cost += test_cases_costs[selected_test_case]
        final_test_suite_costs.append(final_test_suite_cost)
            
        #compute the total failure rate
        if bootqa_program != "elevator2":
            final_effectiveness = 0
            for selected_test_case in final_selected_cases:
                final_effectiveness += test_cases_rates[selected_test_case]
            final_effectivenesses.append(final_effectiveness)
        else:
            final_pcount = 0
            for selected_test_case in final_selected_cases:
                final_pcount += test_cases_pcount[selected_test_case]
            final_pcounts.append(final_pcount)
            
            final_dist = 0
            for selected_test_case in final_selected_cases:
                final_dist += test_cases_dist[selected_test_case]
            final_dists.append(final_dist)
    
    print("Final Test Suite: " + str(final_selected_cases))
    #compute the qpu access times
    qpu_run_times_without_zeros = []
    for access_time in run_times_dictionary[bootqa_program]:
      if access_time != 0:
        qpu_run_times_without_zeros.append(access_time)
    lower_bound, upper_bound = bootstrap_confidence_interval(qpu_run_times_without_zeros, 1000, 0.95)
    for i in range(len(run_times_dictionary[bootqa_program])):
      if run_times_dictionary[bootqa_program][i] == 0:
          run_times_dictionary[bootqa_program][i] = upper_bound
    average_qpu_access_time = statistics.mean(run_times_dictionary[bootqa_program]) 
    
    if bootqa_program == "elevator2":
        var_names = ["final_test_suite_costs", "final_pcounts", "final_dists",
                 "average_qpu_access_time(ms)", "stdev_qpu_access_time(ms)", "all_qpu_access_times(ms)",
                 "qpu_lower_bound(ms)", "qpu_upper_bound(ms)", "qpu_run_times(ms)"]
        values = [final_test_suite_costs, final_pcounts, final_dists, average_qpu_access_time, 
                  statistics.stdev(run_times_dictionary[bootqa_program]), run_times_dictionary[bootqa_program],
                  lower_bound, upper_bound, run_times_dictionary[bootqa_program]]
    else:
        var_names = ["final_test_suite_costs", "final_effectivenesses",
                     "average_qpu_access_time(ms)", "stdev_qpu_access_time(ms)", "all_qpu_access_times(ms)",
                     "qpu_lower_bound(ms)", "qpu_upper_bound(ms)", "qpu_run_times(ms)"]
        values = [final_test_suite_costs, final_effectivenesses, average_qpu_access_time, 
                  statistics.stdev(run_times_dictionary[bootqa_program]), run_times_dictionary[bootqa_program],
                  lower_bound, upper_bound, run_times_dictionary[bootqa_program]]
    
    # Ensure the directory exists
    output_dir = "results/selectqaoa/depolarizing_sim/05"
    os.makedirs(output_dir, exist_ok=True)
    
    # Path to save the file
    file_path = os.path.join(output_dir, f"{bootqa_program}.csv")
    
    # Writing results to the file
    with open(file_path, "w", newline="") as file:
        writer = csv.writer(file)
        writer.writerow(var_names)
        writer.writerow(values)
    print(f"Results saved to {file_path}")

## Effectiveness Empirical Evaluations

Two compare the Single-Objective SelectQAOA with SelectQA, BootQA, and IGDec-QAOA in terms of effectiveness, we compared the solutions by evaluating the **execution times** and **failure rates** of the test suites obtained by the four strategies.

We statistically analyzed the results obtained over ten independent executions by applying the **Mann-Whitney U test** with a **significance level** set at **0.05**. The null hypothesis represents a non-relevant difference between the two approaches.

In contrast, if the null hypothesis is rejected, the magnitude of the difference is computed using the **Vargha-Delaney effect size** $\hat{A}_{12}$ (**we reject the null hypothesis for** p-values $< 0.05$.


In [ ]:
def a12effs(lst1,lst2,rev=True):
  "Computing the Vargha-Delaney's A12 Effect Size"
  more = same = 0.0
  for x in lst1:
    for y in lst2:
      if x==y : same += 1
      elif rev and x > y : more += 1
      elif not rev and x < y : more += 1
  return (more + 0.5*same)  / (len(lst1)*len(lst2))

def stat_test(app1, app2, alt):
    """Computing the Mann-Whitney's U test and the Vargha-Delaney's A12 Effect Size"""
    statistic, pvalue = mannwhitneyu(app1, app2, alternative=alt)

    # Calculate the A12 effect size using Vargha and Delaney's formula
    a12_effect_size = a12effs(app1, app2)

    return pvalue, a12_effect_size

def complete_plotter(data_name, bootqa_costs, qaoa_costs, qtcs_costs, bootqa_rates, qaoa_rates, qtcs_rates):
    """This funcrtion plots the resultying subtestsuites of the compared algorithms"""
    # Create a new figure
    plt.figure(figsize=(10, 6))

    # Plot isolated points for each dataset
    plt.scatter(bootqa_costs, bootqa_rates, label='BootQA', marker='o', color='b', s=50)
    plt.scatter(qaoa_costs, qaoa_rates, label='SelectQAOA', marker='x', color='g', s=200)
    plt.scatter(qtcs_costs, qtcs_rates, label='SelectQA', marker='s', color='r', s=50)

    # Add labels and title
    plt.xlabel('Cost')
    plt.ylabel('Rate')
    plt.title(f'Cost vs Rate for {data_name}')

    # Add a legend
    plt.legend()

    # Show the plot
    plt.grid(True)
    plt.show()

In [ ]:


from matplotlib import cm

data_names = ["gsdtsr", "paintcontrol", "iofrol", "elevator", "elevator2"]
selectqaoa_configs = ["statevector_sim", "aer_sim", "fake_vigo", "depolarizing_sim/01", "depolarizing_sim/02", "depolarizing_sim/05"]
igdec_qaoa_elevator2_configs = ["ideal/qaoa_1/elevator_three", "noise/qaoa_1/elevator_one"]
igdec_qaoa_configs = ["ideal", "noise"]

def read_field_as_list(filepath, field):
    try:
        df = pd.read_csv(filepath)
        return [ast.literal_eval(x) for x in df[field].dropna()]
    except Exception as e:
        print(f"Errore leggendo {field} da {filepath}: {e}")
        return []

def test_normality(data_lists, label):
    for i, lst in enumerate(data_lists):
        if len(lst) > 2:
            flat = np.hstack(lst) if isinstance(lst[0], list) else lst
            stat, p = shapiro(flat)
            mean = np.mean(flat)
            std = np.std(flat)
            print(f"{label} - Media: {mean:.3f}, Std: {std:.3f} - {'Normale' if p > 0.05 else 'NON normale'} (p={p:.3f})")
        else:
            print(f"{label} - Non testabile, lista troppo corta")

def get_color_map(labels):
    unique_labels = list(set(labels))
    cmap = cm.get_cmap('tab10', len(unique_labels))
    return {label: cmap(i) for i, label in enumerate(unique_labels)}

def analyze_and_plot(data_name):
    costs_list = []
    y_values_list = []
    dists_list = []
    labels = []

    if data_name == "elevator2":
        print("=================="+data_name+"==================")
        for config in selectqaoa_configs:
            print("================== SelectQAOA ==================")
            path = f"./results/selectqaoa/{config}/{data_name}.csv"
            costs = read_field_as_list(path, "final_test_suite_costs")
            pcounts = read_field_as_list(path, "final_pcounts")
            dists = read_field_as_list(path, "final_dists")
            dists_list.extend(dists)
            costs_list.extend(costs)
            y_values_list.extend(pcounts)
            labels.extend([f"selectqaoa-{config}"] * len(costs))
            test_normality(costs, f"{config} costs")
            test_normality(pcounts, f"{config} pcounts")
            test_normality(dists, f"{config} dists")

        for config in igdec_qaoa_elevator2_configs:
            print("================== IgDec_QAOA ==================")
            path = f"./results/igdec_qaoa/{config}/size_7/10/solution.csv"
            costs = read_field_as_list(path, "final_test_suite_costs")
            pcounts = read_field_as_list(path, "final_suite_pcounts")
            dists = read_field_as_list(path, "final_suite_dists")
            dists_list.extend(dists)
            costs_list.extend(costs)
            y_values_list.extend(pcounts)
            labels.extend([f"igdecqaoa-{config}"] * len(costs))
            test_normality(costs, f"{config} costs")
            test_normality(pcounts, f"{config} suite_pcounts")
            test_normality(dists, f"{config} suite_dists")

        fig = plt.figure(figsize=(10, 7))
        ax = fig.add_subplot(111, projection='3d')
        color_map = get_color_map(labels)
        
        for cost, pcount, dist, label in zip(costs_list, y_values_list, dists_list, labels):
            if len(cost) == len(pcount) == len(dist):
                ax.scatter(cost, pcount, dist, color=color_map[label], label=label)
        
        # Etichette assi
        ax.set_xlabel("Cost")
        ax.set_ylabel("PCount")
        ax.set_zlabel("Dist", labelpad=10)  # Maggiore padding sull'asse Z
        
        # Migliora visibilità della legenda
        handles, legend_labels = ax.get_legend_handles_labels()
        unique = dict(zip(legend_labels, handles))
        ax.legend(unique.values(), unique.keys(), fontsize='small', loc='upper left', bbox_to_anchor=(1.05, 1), borderaxespad=0.)
        
        plt.title("3D Plot - elevator2", pad=20)
        plt.tight_layout()
        plt.show()

    else:
        print("=================="+data_name+"==================")
        for config in selectqaoa_configs:
            print("================== SelectQAOA ==================")
            path = f"./results/selectqaoa/{config}/{data_name}.csv"
            costs = read_field_as_list(path, "final_test_suite_costs")
            if data_name == "elevator":
                effect = read_field_as_list(path, "final_effectivenesses")
            else:
                effect = read_field_as_list(path, "final_effectivenesses")
            costs_list.extend(costs)
            y_values_list.extend(effect)
            labels.extend([f"selectqaoa-{config}"] * len(costs))
            test_normality(costs, f"{config} costs")
            test_normality(effect, f"{config} effectiveness")

        for config in igdec_qaoa_configs:
            print("================== IgDec_QAOA ==================")
            if data_name == "elevator":
                subpath = f"{config}/qaoa_1/elevator_two"
                field = "final_suite_input_divs"
            elif data_name == "iofrol":
                subpath = f"{config}/qaoa_1/iofrol"
                field = "final_failure_rates"
            else:
                subpath = f"{config}/qaoa_1/{data_name}"
                field = "final_failure_rates"
            path = f"./results/igdec_qaoa/{subpath}/size_7/10/solution.csv"
            costs = read_field_as_list(path, "final_test_suite_costs")
            effectiveness = read_field_as_list(path, field)
            costs_list.extend(costs)
            y_values_list.extend(effectiveness)
            labels.extend([f"igdecqaoa-{config}"] * len(costs))
            test_normality(costs, f"{config} costs")
            test_normality(effectiveness, f"{config} {field}")

        if data_name not in ["elevator2", "elevator", "iofrol"]:
            for algo in ["selectqa", "bootqa"]:
                print("=================="+ algo +"==================")
                path = f"./results/{algo}/{data_name}.csv"
                costs = read_field_as_list(path, "final_test_suite_costs")
                rates = read_field_as_list(path, "final_failure_rates")
                costs_list.extend(costs)
                y_values_list.extend(rates)
                labels.extend([f"{algo}"] * len(costs))
                test_normality(costs, f"{algo} costs")
                test_normality(rates, f"{algo} failure_rates")

        # Plot 2D
        plt.figure()
        color_map = get_color_map(labels)
        
        for cost, yval, label in zip(costs_list, y_values_list, labels):
            color = color_map[label]
            if len(cost) == len(yval):
                plt.scatter(cost, yval, color=color, label=label)
        
        # Elimina label duplicate nella legenda
        handles, labels = plt.gca().get_legend_handles_labels()
        unique = dict(zip(labels, handles))
        plt.legend(unique.values(), unique.keys(), fontsize='small')
        
        plt.title(f"2D Plot - {data_name}")
        plt.xlabel("Cost")
        plt.ylabel("Effectiveness or Rate")
        plt.grid(True)
        plt.tight_layout()
        plt.show()

# Esempio d'uso
for data_name in data_names:
    analyze_and_plot(data_name)


## Efficiency Empirical Evaluation
To compare the efficiency of **SelectQAOA**, **IGDec-QAOA**, **BootQA**, and **SelectQA**, we analyzed their **total run times**.

- **BootQA** executes a **local decomposition** through **bootstrap sampling** (in this work, executed on a **MacBook Air with an M1 Chip and 16GB of RAM**) and directly runs the **Advanced System QPU**.
- **SelectQA** relies solely on the **hybrid_binary_quadratic_model_version2** to handle highly complex optimization problems.

For **BootQA**, we considered the **total run time** as the sum of the **bootstrap sampling process** and **Advanced System QPU execution time**. 

For **SelectQA**, we computed the **total run time** of the **hybrid_binary_quadratic_model_version2**.

**IGDec-QAOA** was runned on a noisy simulator based of **ibm_brisbane**.

The total run time of **SelectQAOA** was computed by summing the **decomposition** and **execution times** of its run on a **Barbora (IT4Innovations supercomputer)** featuring 8 cores of the **Intel Xeon Gold 6240 CPU** with **2.60 GHz**. 

The **empirical reliability** of the findings was statistically validated by analyzing the distribution of the total run times obtained over ten independent runs by each algorithm, using the **Mann-Whitney U test**. The magnitude of the differences between the sequences was quantified using the **Vargha-Delaney effect size** (\(\hat{A}_{12}\)).

In [ ]:
import os
import pandas as pd
import json
from scipy.stats import shapiro
import numpy as np

# Dataset e configurazioni
datasets = ["gsdtsr", "paintcontrol", "iofrol", "elevator", "elevator2"]
selectqaoa_configs = [
    "statevector_sim", "aer_sim", "fake_vigo",
    "depolarizing_sim/01", "depolarizing_sim/02", "depolarizing_sim/05"
]
igdec_qaoa_configs = ["ideal/qaoa_1/", "noise/qaoa_1/"]
igdec_qaoa_elevator2_configs = [
    "ideal/qaoa_1/elevator_three", "noise/qaoa_1/elevator_one"
]

# Funzione per leggere un campo da CSV e convertirlo in lista di float
def read_list_field(path, field):
    try:
        df = pd.read_csv(path)
        value = df[field].iloc[0]
        return eval(value) if isinstance(value, str) else [value]
    except Exception as e:
        print(f"Errore leggendo {field} da {path}: {e}")
        return []

# Funzione per fare il test di Shapiro e stampare statistiche
def analyze_and_report_normality(data_lists, labels, dataset):
    print(f"\n=== Dataset: {dataset} ===")
    for data, label in zip(data_lists, labels):
        if len(data) < 3:
            print(f"{label}: Dati insufficienti per il test di Shapiro.")
            continue
        try:
            stat, p = shapiro(data)
            mean = np.mean(data)
            std = np.std(data, ddof=1)
            status = "Normale" if p > 0.05 else "Non normale"
            print(f"{label}:\n  Media = {mean:.2f} ms\n  Deviazione standard = {std:.2f} ms\n  Normalità = {status} (p = {p:.4f})")
        except Exception as e:
            print(f"{label}: Errore nel test di Shapiro: {e}")

# Loop principale
for dataset in datasets:
    data_lists = []
    labels = []

    # Lettura dati SelectQAOA
    for config in selectqaoa_configs:
        path = f"./results/selectqaoa/{config}/{dataset}.csv"
        times = read_list_field(path, "qpu_run_times(ms)")
        if times:
            data_lists.append(times)
            labels.append(f"SelectQAOA - {config}")

    # Lettura dati IgDec_QAOA
    if dataset == "elevator2":
        for config in igdec_qaoa_elevator2_configs:
            path = f"./results/igdec_qaoa/{config}/size_7/10/solution.csv"
            times = read_list_field(path, "execution_times")
            if times:
                data_lists.append(times)
                labels.append(f"IgDec_QAOA - {config}")
    elif dataset in ["elevator", "iofrol", "gsdtsr", "paintcontrol"]:
        for config in igdec_qaoa_configs:
            if dataset == "elevator":
                path = f"./results/igdec_qaoa/{config}elevator_two/size_7/10/solution.csv"
            else:
                path = f"./results/igdec_qaoa/{config}{dataset}/size_7/10/solution.csv"
            times = read_list_field(path, "execution_times")
            if times:
                data_lists.append(times)
                labels.append(f"IgDec_QAOA - {config.strip('/')}")

    # Dati extra solo per gsdtsr e paintcontrol
    if dataset in ["gsdtsr", "paintcontrol"]:
        # SelectQA
        path = f"./results/selectqa/{dataset}.csv"
        try:
            df = pd.read_csv(path)
            val = float(df["average_qpu_access_time(ms)"].iloc[0])
            data_lists.append([val])
            labels.append("SelectQA - average_qpu_access_time(ms)")
        except Exception as e:
            print(f"Errore leggendo SelectQA per {dataset}: {e}")

        # BootQA
        path = f"./results/bootqa/{dataset}.csv"
        times = read_list_field(path, "exectution_times(ms)")
        if times:
            data_lists.append(times)
            labels.append("BootQA - exectution_times(ms)")

    # Test e statistiche
    analyze_and_report_normality(data_lists, labels, dataset)
